# Type Through The Bible

By Kenneth Burchfiel

Code is released under the MIT license; Bible verses are from the Web English Bible (Catholic Edition)* and are in the public domain.

\* Genesis was not found within the original WEB Catholic Edition folder, so I copied in files from another Web English Bible translation instead. I imagine, but am not certain, that these files are the same as the actual Catholic Edition Genesis files.

# Instructions for getting started:

If you have just downloaded this game, you'll want to create new copies of the **WEB_Catholic_Version_for_game_updated.csv**, **results.csv**, and **word_stats.csv** files. That way, the files will show your results and progress, not mine. You can do so using the following steps:

1. Rename the existing versions of these files as **WEB_Catholic_Version_for_game_updated_sample.csv**, **results_sample.csv**, **character_stats_sample.csv**, and **word_stats_sample.csv**

2. Make a copy of **WEB_Catholic_Version_for_game.csv** and rename it **WEB_Catholic_Version_for_game_updated.csv**.

3. Make a copy of **blank_results_file.csv** and rename it **results.csv**.

5. Make a copy of **blank_word_stats_file.csv** and rename it **word_stats.csv**.


You're now ready to play!

## More documentation to come!

Next steps: (Not necessarily in order of importance)

* Create graphs that show trends in accuracy over time.
* Create graphs that show which dates had the highest average WPMs. (Use a character cutoff of 5000 so that results aren't skewed as much by single races with particularly high or low counts.)
* Finish documenting your v2 print code and the variables/ANSI escape codes that it uses
* Improve chart formatting (e.g. add titles, legend names, etc.)
* Add documentation to other parts of the code as well
* Revise file names or create a renaming script so that users won't need to manually rename them.
* Explain reasoning for removing character stats (essentially, (1) I didn't think these would be that interesting now that I had added in word stats; (2) there were accuracy concerns; and (3) their presence might slow down V2 of the typing test and thus skew the word-level statistics being calculated

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
import time
import plotly.express as px
from getch import getch # Installed this library using pip install py-getch, not
# pip install getch. See https://github.com/joeyespo/py-getch
import numpy as np
from datetime import datetime, date, timezone # Based on 
# https://docs.python.org/3/library/datetime.html
import os
from colorama import just_fix_windows_console, Fore, Back, Style
# From https://github.com/tartley/colorama/blob/master/demos/demo01.py
just_fix_windows_console() # From https://github.com/tartley/colorama/blob/master/demos/demo01.py


In [2]:
extra_analyses = False

Checking whether the program is currently running on a Jupyter notebook:

(The program normally uses getch() to begin typing tests; however, I wasn't able to enter input after getch() got called within a Jupyter notebook and thus couldn't begin a typing test in that situation. Therefore, the program will use input() instead of getch() to start tests when running within a notebook.)

In [3]:
# The following method of determining whether the code is running
# within a Jupyter notebook is based on Gustavo Bezerra's response
# at https://stackoverflow.com/a/39662359/13097194 . I found that
# just calling get_ipython() was sufficient, at least on Windows and within
# Visual Studio Code.

try: 
    get_ipython()
    run_on_notebook = True
except:
    run_on_notebook = False

# print(run_on_notebook)

In [4]:
df_Bible = pd.read_csv('WEB_Catholic_Version_for_game_updated.csv', index_col = 'Verse_Order')
df_Bible.sort_index(inplace = True) # Helps ensure that
# any code that relies on df_Bible's being sorted from the first to the last
# verse will run correctly

df_Bible

,Book_Order,Book_Name,Chapter_Name,Book_and_Chapter,Chapter_Order,Verse_#,Verse,Characters,Typed,Tests,Fastest_WPM,Characters_Typed,Total_Characters_Typed,Count
Verse_Order,,,,,,,,,,,,,,
1,1,GEN,1,GEN 1,1,1,"In the beginning, God created the heavens and ...",56,1,8,193.251509,56,448,1
2,1,GEN,1,GEN 1,1,2,The earth was formless and empty. Darkness was...,135,1,2,140.461827,135,270,1
3,1,GEN,1,GEN 1,1,3,"God said, ""Let there be light,"" and there was ...",52,1,1,84.008493,52,52,1
4,1,GEN,1,GEN 1,1,4,"God saw the light, and saw that it was good. G...",85,1,1,123.198378,85,85,1
5,1,GEN,1,GEN 1,1,5,"God called the light ""day"", and the darkness h...",119,1,1,135.664272,119,119,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35375,95,REV,22,REV 22,1328,17,"The Spirit and the bride say, ""Come!"" He who h...",160,0,0,NaN,0,0,1
35376,95,REV,22,REV 22,1328,18,I testify to everyone who hears the words of t...,159,0,0,NaN,0,0,1
35377,95,REV,22,REV 22,1328,19,If anyone takes away from the words of the boo...,174,0,0,NaN,0,0,1


In [5]:
df_results = pd.read_csv('results.csv', index_col='Test_Number')
df_results

,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms)
Test_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023,10,0,1,2023-10-24,17,2,2,1,2023-10-24 00:17:44.588771456,2023,10,2023-10-24,0,17,2,2,1,NaN,3942,7.553366,NaN,NaN,NaN,NaN,NaN
2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:20.163038180,2023,10,2023-10-24,0,18,2,2,1,NaN,3386,20.595825,NaN,NaN,NaN,NaN,NaN
3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:29.095581096,2023,10,2023-10-24,0,18,2,2,1,NaN,1462,65.728360,NaN,NaN,NaN,NaN,NaN
4,1.698121e+09,2023-10-24 00:21:24.331389,2023-10-24 04:21:24.331389+00:00,56,4.878704,11.478458,137.741497,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,124.569,2023,10,0,1,2023-10-24,21,3,2,1,2023-10-24 00:21:29.210093071,2023,10,2023-10-24,0,21,2,3,1,NaN,1848,56.673704,NaN,NaN,NaN,NaN,NaN
5,1.698121e+09,2023-10-24 00:22:03.749152,2023-10-24 04:22:03.749152+00:00,135,19.078445,7.076048,84.912580,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,116.638,2023,10,0,1,2023-10-24,22,3,2,1,2023-10-24 00:22:22.827597200,2023,10,2023-10-24,0,22,2,3,1,NaN,4204,1.407460,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4259,1.701995e+09,2023-12-07 19:31:17.872645,2023-12-08 00:31:17.872645+00:00,150,13.823856,10.850807,130.209688,NUM,4,3749,"When the camp moves forward, Aaron shall go in...",5.0,130.597686,128.606075,133.982998,133.199,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:31:31.696501200,2023,12,2023-12-07,19,31,3,4,2,0.0,2404,43.631246,9.333333,9.333333,NaN,NaN,NaN
4260,1.701995e+09,2023-12-07 19:31:32.552442,2023-12-08 00:31:32.552442+00:00,84,7.631986,11.006309,132.075712,NUM,4,3750,"put a covering of sealskin on it, spread a blu...",6.0,134.096069,128.857954,133.936306,133.199,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:31:40.184428100,2023,12,2023-12-07,19,31,3,4,2,0.0,2263,46.938776,3.571429,2.380952,NaN,NaN,NaN
4261,1.701996e+09,2023-12-07 19:31:41.216187,2023-12-08 00:31:41.216187+00:00,185,19.186214,9.642340,115.708082,NUM,4,3751,"""On the table of show bread they shall spread ...",7.0,132.705476,128.780425,133.934766,133.195,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:32:00.402400800,2023,12,2023-12-07,19,32,3,4,2,0.0,3389,20.525452,10.810811,10.810811,NaN,NaN,NaN


In [6]:
df_word_stats = pd.read_csv('word_stats.csv')
df_word_stats

,word,typed_word_without_mistakes,word_duration (ms),Count,characters,CPS,WPM
0,In,1,261.6661,1,2,7.643329,91.719944
1,all,1,329.9691,1,3,9.091760,109.101125
2,the,1,158.9050,1,3,18.879205,226.550455
3,land,1,234.7889,1,4,17.036581,204.438966
4,of,1,133.5537,1,2,14.975250,179.702996
...,...,...,...,...,...,...,...
9466,which,1,327.8958,1,5,15.248747,182.984960
9467,they,1,231.6682,1,4,17.266073,207.192873
9468,minister,1,553.3430,1,8,14.457579,173.490945
9469,to,1,183.4473,1,2,10.902314,130.827764


In [7]:
for column in ['Local_Start_Time', 'UTC_Start_Time']:
    df_results[column] = pd.to_datetime(df_results[column])
df_results['WPM'] = df_results['WPM'].astype('float') # Prevents a glitch
# that can be caused when this column is stored as an object. The WPM
# column should only be in object format when the results table is blank.
df_results

,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms)
Test_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023,10,0,1,2023-10-24,17,2,2,1,2023-10-24 00:17:44.588771456,2023,10,2023-10-24,0,17,2,2,1,NaN,3942,7.553366,NaN,NaN,NaN,NaN,NaN
2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:20.163038180,2023,10,2023-10-24,0,18,2,2,1,NaN,3386,20.595825,NaN,NaN,NaN,NaN,NaN
3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:29.095581096,2023,10,2023-10-24,0,18,2,2,1,NaN,1462,65.728360,NaN,NaN,NaN,NaN,NaN
4,1.698121e+09,2023-10-24 00:21:24.331389,2023-10-24 04:21:24.331389+00:00,56,4.878704,11.478458,137.741497,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,124.569,2023,10,0,1,2023-10-24,21,3,2,1,2023-10-24 00:21:29.210093071,2023,10,2023-10-24,0,21,2,3,1,NaN,1848,56.673704,NaN,NaN,NaN,NaN,NaN
5,1.698121e+09,2023-10-24 00:22:03.749152,2023-10-24 04:22:03.749152+00:00,135,19.078445,7.076048,84.912580,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,116.638,2023,10,0,1,2023-10-24,22,3,2,1,2023-10-24 00:22:22.827597200,2023,10,2023-10-24,0,22,2,3,1,NaN,4204,1.407460,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4259,1.701995e+09,2023-12-07 19:31:17.872645,2023-12-08 00:31:17.872645+00:00,150,13.823856,10.850807,130.209688,NUM,4,3749,"When the camp moves forward, Aaron shall go in...",5.0,130.597686,128.606075,133.982998,133.199,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:31:31.696501200,2023,12,2023-12-07,19,31,3,4,2,0.0,2404,43.631246,9.333333,9.333333,NaN,NaN,NaN
4260,1.701995e+09,2023-12-07 19:31:32.552442,2023-12-08 00:31:32.552442+00:00,84,7.631986,11.006309,132.075712,NUM,4,3750,"put a covering of sealskin on it, spread a blu...",6.0,134.096069,128.857954,133.936306,133.199,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:31:40.184428100,2023,12,2023-12-07,19,31,3,4,2,0.0,2263,46.938776,3.571429,2.380952,NaN,NaN,NaN
4261,1.701996e+09,2023-12-07 19:31:41.216187,2023-12-08 00:31:41.216187+00:00,185,19.186214,9.642340,115.708082,NUM,4,3751,"""On the table of show bread they shall spread ...",7.0,132.705476,128.780425,133.934766,133.195,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:32:00.402400800,2023,12,2023-12-07,19,32,3,4,2,0.0,3389,20.525452,10.810811,10.810811,NaN,NaN,NaN


In [8]:
# # If you accidentally overwrite your Unix_Start_Time values with something else,
# # you can recreate them using UTC_Start_Time values as follows:
# # (This code is based on that shown in
# # https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#from-timestamps-to-epoch )
# df_results['Unix_Start_Time'] = ((df_results['UTC_Start_Time'] - pd.Timestamp(
# "1970-01-01", tz = 'utc')) // pd.Timedelta("1ns") / 1000000000)
# df_results

In [9]:
# If you ever need to drop a particular result,
# you can do so as follows:
# df_results.drop(17, inplace = True)
# df_results.to_csv('results.csv') # We want to preserve the index so as not
# to lose our 'Test_Number' values
# df_results

In [10]:
# Creating an RNG seed:
# In order to make the RNG values a bit more random, the following code will
# derive the RNG seed from the decimal component of the current timestamp.
# This seed will change 1 million times each second.

# Using the decimal component of time.time() to select an RNG seed:
current_time = time.time()
decimal_component = current_time - int(current_time) # This 
# line retrieves the decimal component of current_time. int() is used instead
# of np.round() so that the code won't ever round current_time up prior
# to the subtraction operation, which would return a different value.
# I don't think that converting current_time to an integer (e.g. via
# np.int64(current_time)) is necessary, as int() appears to handle at least 
# some integers larger than 32 bits in size just fine.
decimal_component
random_seed = round(decimal_component * 1000000)
decimal_component, random_seed

(0.6796493530273438, 679649)

In [11]:
rng = np.random.default_rng(random_seed) # Based on
# https://numpy.org/doc/stable/reference/random/index.html?highlight=random#module-numpy.random

[This fantastic answer](https://stackoverflow.com/a/23294659/13097194) by Kevin at Stack Overflow proved helpful in implementing user validation code within this program. 

In [12]:
def select_verse():
    print("Select a verse to type! Enter 0 to receive a random verse\n\
or enter a verse number (see 'Verse_Order column of\n\
the WEB_Catholic_Version.csv spreadsheet for a list of numbers to enter)\n\
to select a specific verse.\n\
You can also enter -2 to receive a random verse that you haven't yet typed\n\
or -3 to choose the first Bible verse that hasn't yet been typed.")
    verses_not_yet_typed = df_Bible.query(
                "Typed == 0").copy().index.to_list()
    while True:
        try:
            response = int(input())
        except:
            print("Please enter an integer corresponding to a particular Bible \
verse or 0 for a randomly selected verse.")
            continue # Allows the user to retry entering a number

        if response == 0:
            return rng.integers(1, 35380) # Selects any verse within the Bible.
            # there are 35,379 verses present, so we'll pass 1 (the first verse)
            # and 35,380 (1 more than the last verse, as rng.integers won't 
            # include the final number within the range) to rng.integers().
        elif response == -2:
            if len(verses_not_yet_typed) == 0:
                print("Congratulations! You have typed all verses from \
the Bible, so there are no new verses to type! Try selecting another option \
instead.")
                continue
            print(f"{len(verses_not_yet_typed)} verses have not yet \
been typed.")
            return rng.choice(verses_not_yet_typed) # Chooses one of these
            # untyped verses at random
        elif response == -3:
            if len(verses_not_yet_typed) == 0:
                print("Congratulations! You have typed all verses from \
the Bible, so there are no new verses to type! Try selecting another option \
instead.")
                continue
            print(f"{len(verses_not_yet_typed)} verses have not yet \
been typed.")
            verses_not_yet_typed.sort() # Probably not necessary, as df_Bible
            # was already sorted from the first to the last verse.
            return verses_not_yet_typed[0]
        
        else:
            if ((response >= 1) 
            & (response <= 35379)): # Making sure that the response is 
                # an integer between 1 and 35,379 (inclusive) so that it 
                # matches one of the Bible verse numbers present:                    
                return response
            else: # Will be called if a non-integer number was passed
                    # or if the integer didn't correspond to a Bible verse
                    # number. 
                print("Please enter an integer between 1 and 35,379.") # Since
                # we're still within a While loop, the user will be returned
                # to the initial try/except block.


In [13]:
column_width = 120
column_width - (55 % column_width)

65

In [14]:
def create_word_table(verse):
    '''Creating a list of words within the verse that we can use for
    word-based typing test analyses:
    We could use verse.split(' ') to create a list of words. However,
    since our goal is to build a list that can then serve as the basis
    of analyses, we'll want to create a list that includes:
    1. The starting and finishing index of each word within the verse. This will
    allow us to determine when the player has arrived at and completed
    that word.
    2. Words without any spacing or punctuation attached. (Learning how fast
    you wrote 'they' is more interesting than is learning how fast 
    you wrote '"that' or 'that,'.)
    Therefore, the following code is more complex than a simple split()
    operation.
    This function also counts contractions ('don't', 'can't,
    etc.) as individual words. Some care is needed to distinguish between
    apostrophes/single quotes that begin contractions and those that start
    or end single quotes.'''

    first_letter_index_list = []
    # Determining the indices of the verse that contain starting letters of
    # words: 
    # (We can find these indices by searching for characters that are preceded
    # by a non-alphabetic character *or* an alphabetic character that is also
    # the first character in the verse.
    for i in range(1, len(verse)): # Starting at 1 rather than 0 because
        # we'll sometimes need to reference the preceding character
        # (which wouldn't be possible if i were 0)

        if i == 1:
            if ((verse[i].isalpha()) & (verse[i-1].isalpha() == False)): 
                # This if statement addresses a special case
                # in which a verse begins with ' and a letter. In this 
                # case, because there are no letters preceding the ' 
                # (since i is 1), we'll assume that the ' is the start of 
                # a single quote rather than the beginning of a contraction
                # and will thus add the letter to our starting letters list.
                # print("Found first letter of word (Part A)")
                first_letter_index_list.append(i)
                continue # Now that we've added i to the list, we can 
                # return to the top of the for loop. (This will prevent the
                # code from adding duplicate copies of a starting letter
                # to first_letter_index_list.

            if (verse[i-1].isalpha()):
                # print("Found first letter of word (Part B)")
                first_letter_index_list.append(i-1) # In this case, the
            # character preceding i is the starting letter 
            # rather than i itself (which may or may not be a letter).
                continue

        if ((verse[i].isalpha()) & 
        ((verse[i-1].isalpha() == False) & (verse[i-1] != "'"))): # If 
            # the current character is a letter (e.g. isalpha() is True)
            # and the character behind it isn't a letter *and* also isn't
            # "'" (which is found in contractions), we'll assume that we
            # have found the start of a word.
            # This if statement will fail to detect the first word in a verse
            # if it gets preceded by a single quote (e.g. 'Hello' in 
            # 'Hello there'), but a previous if statement
            # will have already detected it.
            # print("Found first letter of word (Part C)")
            first_letter_index_list.append(i)
            continue
    
        if i >= 2:
            if ((verse[i].isalpha()) & 
            ((verse[i-1].isalpha() == False)) &
            (verse[i-2].isalpha() == False)): # In this case,
                # if both of the characters preceding a letter
                # aren't letters, then we'll assume that
                # this letter begins a word. (This was added in
                # to account for words that are preceded by a single
                # quote. Checking the character before that single quote
                # helps us distinguish apostrophes that begin quotes (which
                # won't be preceded by a letter)
                # from apostrophes that form part of contractions (which
                # will be).
                # print("Found first letter of word (Part D)")
                first_letter_index_list.append(i)
                continue


    # print(first_letter_index_list)


    # Now that we've determined the index of each word's starting letter,
    # we will go through the verse again to locate the final letter
    # within each word.
    df_word_index_list = []

    for index in first_letter_index_list: # This code iterates through
        # each word's starting character.
        first_letter = verse[index]
        # Initializing the word started by this character with 
        # the starting character:
        word = first_letter
        i = 0
        while True:
            i += 1
            position_within_verse = index + i
            # print("i is now", i)
            if position_within_verse < (len(verse) -1): # If this is False,
                # then there are at least two characters left in the verse.
                # We need to perform this check so that an upcoming if
                # statement that reads two characters ahead won't go
                # out of bounds.
                next_character = verse[position_within_verse]
                # print(f"Next character: '{next_character}'")
                if (next_character.isalpha() == True) | (
                    (next_character == "'") & 
                    (verse[position_within_verse + 1].isalpha() == True)): 
                    # Here, the next character will be added to the word if
                    # (1) it's a letter or (2) it's an apostrophe followed
                    # by a letter (which indicates that the apostrophe
                    # is part of a contraction).
                    word += next_character
                else:
                    last_letter_index = position_within_verse -1
                    break # Now that we've found the last letter, we can
                    # exit out of the while loop and begin to evaluate
                    # the next first index in first_letter_index_list.

            elif position_within_verse < len(verse): # In this case, 
                # we haven't yet made it to the very end of the verse.
                # We need to use elif here rather than 'if' so that
                # the preceding block of if statements and this one
                # will not both return True.
                next_character = verse[position_within_verse]
                if next_character.isalpha() == True: # 
                    word += next_character
                else:
                    last_letter_index = position_within_verse -1
                    break
            else: # In this case, we're already at the end of the verse,
                # so we can instead store the previous index position
                # within last_letter_index.
                last_letter_index = position_within_verse -1
                break

        df_word_index_list.append({'first_letter_index':index, 
            'last_letter_index': last_letter_index,
            'word':word})
        # print(index, last_letter_index, word)

    df_word_index_list = pd.DataFrame(df_word_index_list)
    df_word_index_list['previous_character_index'] = np.where(
        df_word_index_list['first_letter_index'] != 0,  
        df_word_index_list['first_letter_index'] - 1, np.NaN)
    df_word_index_list
    # print("Words identified by create_word_table():",df_word_index_list['word'].to_list())
    return df_word_index_list

In [15]:
def run_typing_test(verse_order, results_table, 
    word_stats_table, test_type = 'v2'):
    '''This function calculates how quickly the user types the characters
    passed to the Bible verse represented by verse_order, then saves those 
    results to the DataFrame passed to results_table.'''

    # Retrieving the verse to be typed:
    # df_Bible uses verse order values for its index, so we can access
    # verses within df_Bible by passing verse_order to .at[].
    verse = df_Bible.at[verse_order, 'Verse']
    book = df_Bible.at[verse_order, 'Book_Name']
    chapter = df_Bible.at[verse_order, 'Chapter_Name']
    verse_number_within_chapter = df_Bible.at[verse_order, 'Verse_#']
    
    df_word_index_list = create_word_table(verse)
    # print(df_word_index_list)

    # I moved these introductory comments out of the following while loop
    # in order to simplify the dialogue presented to users during retest
    # attempts.
    print(f"Welcome to the typing test! Your verse to type is {book} \
{chapter}:{verse_number_within_chapter} (verse {verse_order} \
within the Bible .csv file).\n")
    if run_on_notebook == False:
        print("Press any key to begin typing!")
    else:
        print("Press Enter to begin the test!")
    
    complete_flag = 0
    while complete_flag == 0:
        print(f"Here is the verse:\n\n{verse}") 

        if run_on_notebook == False: # In this case, we can use getch()
            # to begin the test.
        # time.sleep(3) # I realized that players could actually begin typing
        # during this sleep period, thus allowing them to complete the test
        # faster than intended. Therefore, I'm now having the test start
        # after the player hits a character of his/her choice. getch()
        # accomplishes this task well.
        # A simpler approach would be to add in an additional input block
        # and have the player begin after he/she presses Enter, but that would
        # cause the player's right hand to leave the default home row position,
        # which could end up slowing him/her down. getch() allows any character
        # to be pressed (such as the space bar) and thus avoids this issue.

            start_character = getch() # See https://github.com/joeyespo/py-getch
        
        else: # When running the program within a Jupyter notebook, I wasn't
            # able to enter input after getch() was called, so I created
            # an alternative start method below that simply uses input().
            input()

        # The following line determines the width of the terminal just before
        # the beginning of the typing test. This width will help determine
        # when a line has been completed, which will in turn inform
        # when to move the cursor up and how many lines to fill with
        # blank spaces.

        if run_on_notebook == False: # The following line crashed for me
            # when running the program within a notebook.
            column_width = os.get_terminal_size().columns
        # get_terminal_size() is cross-platform. See
        # https://docs.python.org/3.8/library/os.html?highlight=get_terminal_size#os.get_terminal_size
        else:
            column_width = 120 # The default column width for my
            # terminal

        # print(f"Column width is {column_width}")
        print("Start!")      
        if test_type == 'v1': 
            # This is a simple typing test setup that receives input from
            # the user when 'Enter' is pressed, then checks that input
            # against the verse. Because it doesn't check the response
            # for accuracy as the player types, the player might not realize
            # a character was mistyped until the very end, which can get
            # frustrating. Therefore, I've now added in a new version
            # of the test (called 'v2') that can be used instead. 
            no_mistakes = np.NaN
            backspaces_as_pct_of_length = np.NaN
            incorrect_characters_as_pct_of_length = np.NaN
            unix_start_time = time.time()
            local_start_time = pd.Timestamp.now()
            utc_start_time = pd.Timestamp.now(timezone.utc)
            # I used to use ISO8601-compatible timestamps via the following
            # lines, but decided to switch to a value that Pandas would 
            # immediately recognize as a datetime.
            # local_start_time = datetime.now().isoformat()
            # utc_start_time = datetime.now(timezone.utc).isoformat()

            typing_start_time = time.perf_counter_ns()
            verse_response = input() 
            # The following code will execute once the player finishes typing and
            # hits Enter. (Having the program evaluate the player's entry only after
            # 'Enter' is pressed isn't the best option, as the time required to
            # hit Enter will reduce the player's reported WPM. Version v2, 
            # shown below, stops the test right when the final correct
            # character is typed, which will make the final WPM slightly faster.
            typing_end_time = time.perf_counter_ns()
            typing_time = (typing_end_time - typing_start_time) / 1000000000

        elif test_type == 'v2':
            # This version of the test checks the player's input after
            # each character is typed. If the player's input is correct
            # so far, the text will be highlighted green; otherwise,
            # it will be highlighted red. (This allows the player to be 
            # notified of an error without the need for a line break
            # in the console, which could prove distracting.)
            # This function has been tested on Windows, but not yet 
            # on Mac or Linux. The use of the Colorama library should
            # make it cross-platform, however.
            verse_response = '' # This string will store the player's 
            # response.
            no_mistakes = 1 # This flag will get set to 0 if the player makes
            # a mistake. If it remains at 1 throughout the race, then
            # a mistake-free race will get logged in results_table.
            previous_line_count = 1
            backspace_count = 0
            incorrect_character_count = 0
            word_stats_list = []
            # code_execution_time_list = []

            # The following two lists will keep track of first- and last-letter
            # indices that have already been reached by the player. These lists
            # will be incorporated into if statements so that (1) timing for a
            # word will begin only if that word's first letter index is not in 
            # first_letter_indices_reached and (2) timing for a word will end
            # only if that word's last letter index is not in
            # last_letter_indices_reached. This will prevent duplicate 
            # timing entries for the same word from getting added to the word
            # stats list; in addition, it will prevent the player from
            # 'resetting' the timing for a given word by returning to its index.
            first_letter_indices_reached = []
            last_letter_indices_reached = []

            first_keypress = 1 # Designates whether or not the player is 
            # currently making his or her first keypress. This flag, which
            # will be set to 0 after the first keypress is made,
            # will help determine whether or not to begin timing the player's
            # first word.
            last_letter_index = -1 # Initializing this variable with a number
            # that will never occur within the game so that this value won't
            # get interpreted as an actual word starting point.

            unix_start_time = time.time()
            local_start_time = pd.Timestamp.now()
            utc_start_time = pd.Timestamp.now(timezone.utc)
            typing_start_time = time.perf_counter_ns()
            # Using perf_counter_ns() allows for more accurate time duration
            # calculations than does time.time(). See
            # https://docs.python.org/3/library/time.html#time.perf_counter_ns


            while True: # This while loop allows the player to 
                # enter multiple characters.
                # The following if statement determines whether to 
                # begin timing the player's first word. Timing will only
                # begin
                if (len(verse_response) == 0) & (
                    0 in df_word_index_list['first_letter_index'].to_list()
                    ) & (first_keypress == 1):
                    word_start_time = typing_start_time
                    last_letter_index = df_word_index_list.query(
                        'first_letter_index == 0').copy().iloc[0][
                            'last_letter_index']
                    word = df_word_index_list.query(
                        'first_letter_index == 0').copy().iloc[0][
                            'word']
                    first_letter_indices_reached.append(0)
                    # print(f" Started typing {word}.")
                    typed_word_without_mistakes = 1
                    # print(f" {last_letter_index}")
                # code_execution_end_time = time.perf_counter_ns()
                # if first_keypress == 0: 
                #     code_execution_time_list.append((code_execution_end_time - character_press_time) / 1000000)
                    # We divide by 1,000,000 to convert from nanoseconds (the output
                    # of perf_counter_ns() to milliseconds.
                character = getch() # getch() allows each character to be 
                # checked, making it easier to identify mistyped words.
                character_press_time = time.perf_counter_ns()
                first_keypress = 0

                if character == b'\x08': 
                    # This will return True if the user hits backspace.
                    # In this case, we'll want to remove the latest character
                    # from verse_response in order to keep that value
                    # in sync with what the player sees on the screen.
                    # Calling print(character) after
                    # hitting backspace revealed that b'\x08' was the code
                    # associated with the backspace key. 
                    backspace_count += 1
                    verse_response = verse_response[:-1] # Trims the last
                    # value off verse_response.
                    typed_word_without_mistakes = 0
                elif character == b'`':
                    print(Style.RESET_ALL) # Resets the color of the text.
                    verse_response += character.decode('ascii') # The presence
                    # of this character within verse_response will instruct
                    # the program to exit the user out of this test later on.
                    # See https://pypi.org/project/colorama/
                    break
                else: 
                    # The following line adds the latest character typed
                    # to verse_response.
                    try:
                        verse_response += character.decode('ascii')  
                        # See https://stackoverflow.com/questions/17615414/how-to-convert-binary-string-to-normal-string-in-python3
                    except: # Keys that fall out of the ascii subset, such as 
                        # arrow keys, could cause the above line to crash. Therefore,
                        # when the above line fails to work, the following 'continue'
                        # statement will allow the program to ignore the key and move
                        # back to the beginning of the loop.
                        continue

                # Determining which color to use for the text:
                if verse[0:len(verse_response)] == verse_response: # If this returns
                    # True, the player's response is correct so far.
                    text_color = Fore.GREEN
                    # Adding the time it took to type the last character
                    # to the list: (Note that the time it takes to 
                    # enter a backspace won't be included.)
                    verse_response_minus_one = len(verse_response) -1 # The character
                    # Index values in df_word_index_list start at 0, so this variable
                    # will help us convert between verse lengths and index positions.        
                    
                    if character != b'\x08':

                        # Checking whether a word has begun or ended:
                        # We're placing these checks within the correct response and
                        # no backspace clauses so that a typo or backspace won't
                        # count as having correctly begun or ended a word.
                        # Note that last_letter_indices_reached and 
                        # first_letter_indices_reached will be checked for the
                        # presence of verse_response_minus_one. If it's already
                        # there, then no action will be taken, as timing
                        # for that word has already ended or started.

                        # Checking whether the player has finished typing a word:
                        if (verse_response_minus_one == last_letter_index) & (
                            last_letter_index not in last_letter_indices_reached):
                            word_end_time = character_press_time
                            # print(f"Finished typing {word} in \
# {(word_end_time - word_start_time) / 1000000} ms. typed_word_without_mistakes \
# is set to {typed_word_without_mistakes}.")
                            word_stats_list.append({"word":word, 
                            "word_duration (ms)": (word_end_time - 
                            word_start_time) / 1000000, 
                            "typed_word_without_mistakes":typed_word_without_mistakes})
                            last_letter_indices_reached.append(last_letter_index)
                            # Other analyses can be added to our 
                            # word stats table later on, so we don't
                            # need to compute them now.

                        # Checking whether the player has started typing a word:
                        if (verse_response_minus_one in df_word_index_list[
                            'previous_character_index'].to_list()) & (
                            verse_response_minus_one not in first_letter_indices_reached):
                            # If this returns true, we know we're
                            # at the starting point of a new word.
                            # print(verse_response_minus_one, df_word_index_list['previous_character_index'])
                            # print("Start of new word detected (Point A).")
                            typed_word_without_mistakes = 1
                            verse_response_minus_one = len(verse_response) -1
                            word_start_time = character_press_time # The start time of 
                            # this new word is defined as the time that the character 
                            # preceding the word was pressed.
                            last_letter_index = df_word_index_list.query(
                            "previous_character_index == @verse_response_minus_one").iloc[
                            0]['last_letter_index']
                            word = df_word_index_list.query(
                            "previous_character_index == @verse_response_minus_one").iloc[
                            0]['word']
                            # print(f" Started typing {word}.")
                            first_letter_indices_reached.append(
                            verse_response_minus_one)

                else:
                    no_mistakes = 0 # This flag will remain at 0 for the 
                    # rest of the race.
                    typed_word_without_mistakes = 0 
                    text_color = Fore.RED 
                    if character != b'\x08': # Backspaces won't be counted
                        # towards the incorrect character count so that
                        # players won't be double-penalized for mistyping
                        # a character.
                        incorrect_character_count += 1
                
                # Printing the player's response so far: 
                
                # This process will involve printing the entirety of verse_response
                # after each character is pressed rather than 
                # just the most recent character. 
                # This code is more complex than a regular print statement, but it has
                # several advantages:
                # 1. It allows the player to quickly determine when a typo has 
                # occurred (as the text will show up in red rather than in green).
                # 2. It supports the use of backspace to correct responses on 
                # previous lines. (I wasn't able to navigate to a previous line 
                # using backspace when printing single characters at a time.)
                # 3. It allows the cursor to always appear to the right of the most
                # recent character. If the latest typed line takes up the entire
                # width of the console, the cursor will appear on the left of 
                # the following line.
                # The development of this code involved a decent amount of trial and 
                # error, but I'll try to explain the function of each line in order to
                # make the final result more intuitive.


                line_count = ((len(verse_response)) // column_width) + 1
                # Calculates the number of lines on which the player's
                # response appears. The inclusion of the max() function ensures
                # that line_count will always be at least 1.
                # 1 is added to verse_response because, if the response has extended
                # to the right side of the terminal, another line will get added
                # in (via code below) to make room for the cursor. Thus, line_count
                # needs to be incremented by 1 in that case to reflect the terminal's
                # output.

                # The following code adjusts to changes in the response's line count.
                # If the line count goes up (as indicated by line_count's exceeding
                # previous_line_count), the response printout will be preceded
                # by a newline so that more space is available to print the longer
                # text. If the line count goes down (e.g. due to a backspace),
                # the the printout will be preceded by an up cursor statement
                # since less space will be needed to print the line.
                if line_count > previous_line_count:
                    line_change_shift_command = '\n'
                elif line_count < previous_line_count: 
                    line_change_shift_command = "\033[A"
                    # "\033[A" is an ANSI escape code that moves the cursor 
                    # up by one line. See
                    # at https://pypi.org/project/colorama/
                else: # No command is necessary if the number of lines is the same 
                    # as before
                    line_change_shift_command = ''

                previous_line_count = line_count 

                # If more than one line is present, we'll need to move the cursor
                # up by the number of lines -1. Otherwise, an extra line will get
                # printed with each character.
                if line_count > 1:
                    up_command = f"\033[{line_count -1}A"
                # This ANSI escape code, based on Richard's response at
                # https://stackoverflow.com/a/33206814/13097194 ,
                # will move the cursor up line_count -1 times.
                else:
                    up_command = '' # If the response is still on the first line,
                #     # there's no need to move the cursor up, as its vertical
                #     # position won't shift in the process of writing the response.

                clear_text_to_right_command = '\033[0K' # Based on
                # https://en.wikipedia.org/wiki/ANSI_escape_code
                # and on https://pypi.org/project/colorama/

                if column_width - (len(verse_response) % column_width) == 1:
                    left_cursor_shift = ''
                else:
                    left_cursor_shift = '\033[D'

                # Printing out various variables related to
                # the subsequent print statement can be useful for debugging.
                # print("\033A",line_count, len(verse_response), column_width, 
                # column_width - (len(verse_response) % column_width) == 1)
                print(f"\r{clear_text_to_right_command}\
{line_change_shift_command}{up_command}{text_color}\
{verse_response} {left_cursor_shift}", end = '')
                # For the use of Colorama to produce red and green text, see
                # https://pypi.org/project/colorama/
                # and https://stackoverflow.com/a/3332860/13097194
                
                if verse_response == verse: # Note that, unlike with version
                    # v1, the player does not need to hit 'Enter' in order
                    # to end the typing test after writing a completed
                    # verse. This should speed up his/her WPM as a result.
                    typing_end_time = time.perf_counter_ns()
                    typing_time = (
                        typing_end_time - typing_start_time) / 1000000000
                    # Dividing by 1 billion to convert typing_time from nanoseconds
                    # to seconds
                    print('\n'*line_count+'Success!') # The cursor
                    # needs to be moved past the lines already printed
                    # so that 'Success' won't overwrite any of the words.
                    print(Style.RESET_ALL)

                    # Accuracy calculations:
                    # Calculating backspaces as a percentage of verse length:
                    backspaces_as_pct_of_length = (
                        100 * backspace_count / len(verse)) # The 100* 
                        # multiplier converts these values from 
                        # proportions to percentages.

                    # Calculating incorrect entries as a percentage of verse
                    # length:
                    incorrect_characters_as_pct_of_length = (
                        100 * incorrect_character_count / len(verse))
                
                    # Calculating timing statistics at the word level:
                    word_stats_for_latest_test = pd.DataFrame(word_stats_list)
                    # print(f"Code execution stats: Median: {np.median(
                    # code_execution_time_list)}, Mean: {np.mean(
                    # code_execution_time_list)}, Max: {max(
                    # code_execution_time_list)}, Min: {min(
                    # code_execution_time_list)}. All execution times: {
                    # code_execution_time_list}")
                    break

        if verse_response == verse:
            print(f"Well done! You typed the verse correctly.")
            complete_flag = 1 # Setting this flag to 1 allows the player to exit
            # out of the while statement.
        elif (verse_response.lower() == 'exit') or ('`' in verse_response):
            print("Exiting typing test.")
            return results_table, word_stats_table # Exits the function without saving the 
            # current test to results_table or df_Bible. This function has
            # been updated to work with both versions of the typing
            # test.
        else:
            print("Sorry, that wasn't the correct input.")   
            # Identifying incorrectly typed words:
            verse_words = verse.split(' ')
            verse_response_words = verse_response.split(' ')[0:len(verse_words)]
            # I added in the [0:len(verse_words)] filter so that the following
            # for loop would not attempt to access more words that were 
            # present in the original verse (which would cause the game
            # to crash with an IndexError).
            for i in range(len(verse_response_words)):
                if verse_response_words[i] != verse_words[i]:
                    print(f"Word number {i} ('{verse_words[i]}') \
was typed '{verse_response_words[i]}'.")
                    # If the response has more or fewer words than the original
                    # verse, some correctly typed words might appear within
                    # this list also.
            print("Try again!") # complete_flag will still be 0 in this case,
            # so the while loop will continue back to the beginning.

    # Calculating typing statistics and storing them within a single-row
    # DataFrame:

    cps = len(verse) / typing_time # Calculating characters per second
    wpm = cps * 12 # Multiplying by 60 to convert from characters to minutes, 
    # then dividing by 5 to convert from characters to words.
    wpm

    if test_type == 'v2':
        word_stats_table = pd.concat(
        [word_stats_table, word_stats_for_latest_test]).reset_index(
        drop = True)

    # Creating a single-row DataFrame that stores the player's results:
    df_latest_result = pd.DataFrame(index = [
        len(results_table)+1], data = {'Unix_Start_Time':unix_start_time, 
    'Local_Start_Time':local_start_time,
    'UTC_Start_Time':utc_start_time,
    'Characters':len(verse),
    'Seconds':typing_time, 
    'CPS': cps,
    'WPM':wpm,
    'Mistake_Free_Test':no_mistakes,
    'Backspaces as % of Verse Length': backspaces_as_pct_of_length,
    'Incorrect Characters as % of Verse Length': \
incorrect_characters_as_pct_of_length,
    'Book': book,
    'Chapter': chapter,
    'Verse #': verse_number_within_chapter,
    'Verse':verse, 
    'Verse_Order':verse_order})
    df_latest_result.index.name = 'Test_Number'
    df_latest_result

    # Adding this new row to results_table:
    results_table = pd.concat([results_table, df_latest_result])
    # Note: I could also have used df.at or df.iloc to add a new row
    # to df_latest_result, but I chose a pd.concat() setup in order to ensure
    # that the latest result would never overwrite an earlier result.
    

    # Rank and percentile data needs to be recalculated after each test,
    # as later results can affect the rank and percentile of earlier results.
    # I could compute these statistics later, but calculating them here
    # allows the player to view his/her statistics after each test.

    results_table['WPM_Rank'] = results_table['WPM'].rank(
    ascending = False, method = 'min').astype('int')
    results_table['WPM_Percentile'] = results_table['WPM'].rank(pct=True)*100
    latest_rank = results_table.iloc[-1]['WPM_Rank']
    # Note: These percentile results may differ from the results
    # calculated by np.quartile later in this function, likely a result of
    # different calculation methodologies. These differences should narrow
    # as more tests are completed.

    latest_percentile = results_table.iloc[-1]['WPM_Percentile'].round(3)
    number_of_tests = len(results_table)
    last_10_avg = results_table['WPM'].rolling(10).mean().iloc[-1]
    
    # The player's rolling 10-race average will be NaN until he/she has
    # completed 10 tests. Therefore, the following if statement will 
    # return a blank last 10 races report unless at least 10 tests
    # are present in results_table.
    if len(results_table) >= 10:
        last_10_report = f' You have averaged \
{last_10_avg.round(3)} WPM over your last 10 tests.' # The space
    # space before 'You' separates this text from the rest of
    # the print statement below.
    else:
        last_10_report = ''

    print(f"Your CPS and WPM were {round(cps, 3)} and {round(wpm, 3)}, \
respectively. Your WPM percentile was {latest_percentile} \
({latest_rank} out of {number_of_tests} tests).{last_10_report}")  

    # Updating df_Bible to store the player's results: (This will allow the
    # player to track how much of the Bible he/she has typed so far)
    df_Bible.at[verse_order, 'Typed'] = 1 # Denotes that this verse
    # has now ben typed. Note that .loc[] would not work here because
    # we are updating a value, not merely retrieving one.
    df_Bible.at[verse_order, 'Tests'] += 1 # Keeps track of how 
    # many times this verse has been typed
    fastest_wpm = df_Bible.at[verse_order, 'Fastest_WPM']
    if ((pd.isna(fastest_wpm) == True) | (wpm > fastest_wpm)): 
        # In these cases, we should replace the pre-existing Fastest_WPM value
        # with the WPM the player just achieved.
        # I found that 5 > np.NaN returned False, so if I only checked for
        # wpm > fastest_wpm, blank fastest_wpm values would never get overwritten.
        # Therefore, I chose to also check for NaN values 
        # in the above if statement.
        df_Bible.at[verse_order, 'Fastest_WPM'] = wpm

    # Autosaving results as separate files: (That way, if the script crashes,
    # the player won't lose all of his/her progress.)
    if verse_order % 10 == 0: # The autosave will only take place for ~10%
        # of the user's verses, thus saving processing time (and wear
        # on the user's SSD, though I'm not sure how much of a difference
        # this would make for the SSD's longevity).
        try:
            results_table.to_csv('df_results_autosave.csv', index = True)
            df_Bible.to_csv('WEB_Catholic_Version_for_game_updated_autosave.csv',
            index = True)
            word_stats_table.to_csv('word_stats_autosave.csv', 
            index = False)
            print("Autosave complete.")
        except:
            print("At least one of the autosave files could not be saved. Close \
    out of any open autosave files before starting the next test \
    so that they can be updated.")
    return (results_table, word_stats_table)


In [16]:
def select_subsequent_verse(previous_verse_order):
    '''This function allows the player to specify which verse to
    type next, or, alternatively, to exit the game.'''
    print("Press 0 to retry the verse you just typed; \
1 to type the next verse; 2 to type the next verse that hasn't yet been typed; \
-3 to select the first verse that hasn't been typed; \
3 to select a different verse; \
-1 to save your results and exit; \
and -2 to save your results without running the analysis \
portion of the script.") # The analysis portion can take a decent amount of
# time to run, which is why an option to save without running these analyses
# was added in. These analyses can then get updated during a later session.
    verses_not_yet_typed = df_Bible.query(
    "Typed == 0").copy().index.to_list()
    while True: 
            try:
                response = int(input())
            except: # The user didn't enter a number.
                print("Please enter a number.")      
                continue
            if response == 0:
                return previous_verse_order
            elif response == 1:
                if previous_verse_order == 35379: # The verse order value
                    # corresponding to the final verse of Revelation
                    print("You just typed the last verse in the Bible, so \
there's no next verse to type! Please enter an option other than 1.\n")
                    continue
                else:
                    return previous_verse_order + 1
            elif response == 2:
                # In this case, we'll retrieve a list of verses that haven't
                # yet been typed; filter that list to include only verses
                # greater than previous_verse_order; and then select
                # the first verse within that list (i.e. the next 
                # untyped verse).
                if len(verses_not_yet_typed) == 0:
                    print("Congratulations! You have typed all verses from \
the Bible, so there are no new verses to type! Try selecting another option \
instead.")
                    continue
                print(f"{len(verses_not_yet_typed)} verses have not yet \
been typed.")
                verses_not_yet_typed.sort() 
                next_untyped_verses = [verse for verse in verses_not_yet_typed 
                if verse > previous_verse_order]
                return next_untyped_verses[0]
            
            elif response == -3:
                if len(verses_not_yet_typed) == 0:
                    print("Congratulations! You have typed all verses from \
the Bible, so there are no new verses to type! Try selecting another option \
instead.")
                    continue
                print(f"{len(verses_not_yet_typed)} verses have not yet \
been typed.")
                verses_not_yet_typed.sort()
                return verses_not_yet_typed[0]

            elif response == 3:
                return select_verse()
            elif response in [-1, -2]:
                return response
            else: # A number other than one of the above options was passed.
                print("Please enter a whole number from -3 to 3 (inclusive).\n")  

In [17]:
def calculate_current_day_results(df):
    ''' This function reports the number of characters, total verses, and 
    unique verses that the player has typed so far today.'''
    df_current_day_results = df[pd.to_datetime(
        df['Local_Start_Time']).dt.date == datetime.today().date()].copy()
    if len(df_current_day_results) == 0:
        result_string = "You haven't typed any Bible verses yet today."
    else:
        characters_typed_today = df_current_day_results['Characters'].sum()
        total_verses_typed_today = len(df_current_day_results)

        # Allowing for both singular and plural versions of 'verse' to 
        # be displayed:
        if total_verses_typed_today == 1:
            total_verses_string = 'verse'
        else:
            total_verses_string = 'verses'

        unique_verses_typed_today = len(df_current_day_results[
            'Verse_Order'].unique())

        if unique_verses_typed_today == 1:
            unique_verses_string = 'verse'
        else:
            unique_verses_string = 'verses'

        average_wpm_today = round(df_current_day_results['WPM'].mean(), 3)
        median_wpm_today = round(df_current_day_results['WPM'].median(), 3)
        result_string = f"So far today, you have typed \
{characters_typed_today} characters from {total_verses_typed_today} Bible \
{total_verses_string} (including {unique_verses_typed_today} unique \
{unique_verses_string}). Your mean and median WPM today are \
{average_wpm_today} and {median_wpm_today}, respectively."
    return result_string

In [18]:
def run_game(results_table, word_stats_table):
    '''This function runs Type Through the Bible by 
    calling various other functions. It allows users to select
    verses to type, then runs typing tests and stores the results in
    the DataFrame passed to results_table.'''
    
    print("Welcome to Type Through the Bible!")
    # The game will now share the player's progress for the current day:
    print(calculate_current_day_results(results_table))

    if run_on_notebook == True: # I haven't been able to get version
        # v2 of the typing test to work within a Jupyter notebook, 
        # so the following line forces notebook-based runs to use version v1.
        typing_test_version = 'v1'
    else: # In this case, the user gets to choose whether to to use 
        # v1 or v2.
        print("To switch to a simpler typing test method that doesn't \
check your input as you type, enter v1. Otherwise, to stick with the \
recommended version, press Enter.")
        response = input()
        if (response == 'v1') or (run_on_notebook == True): # Version 2 likely
            # won't work within Jupyter notebooks, 
            # so the version will always be kept as v1 for notebook users.
            typing_test_version = 'v1'
        else:
            typing_test_version = 'v2'

    # The method for exiting a test in progress differs by typing test
    # version, so the game will now explain how the player can exit out of 
    # his/her version of the test.
    if typing_test_version == 'v1':
        print("Version 1 selected. Note that you can exit a test in \
progress by typing 'exit' and then hitting Enter.")
    if typing_test_version == 'v2':
        print("Version 2 selected. Note that you can exit a test in progress \
by hitting the ` (backtick) key.")
              
              

    verse_order = select_verse()
    

    while True: # Allows the game to continue until the user exits
        results_table, word_stats_table = run_typing_test(
            verse_order=verse_order, 
        results_table=results_table, 
        word_stats_table=word_stats_table,
        test_type = typing_test_version)
        # The game will next share an updated progress report:
        print(calculate_current_day_results(results_table))
        
        # The player will now be prompted to select a new verse order 
        # (or to save and quit).
        verse_order = select_subsequent_verse(
            previous_verse_order=verse_order)
        if verse_order == -1: # In this case, the game will quit and the 
            # user's new test results will be saved to results_table.
            run_analyses = 1
            return (results_table, word_stats_table, run_analyses)
        if verse_order == -2: # In this case, the game will quit and the 
            # user's new test results will be saved to results_table.
            # However, the analysis portion of the script will be skipped 
            # in order to save time.
            run_analyses = 0
            return (results_table, word_stats_table, run_analyses)


In [19]:
df_results, df_word_stats, run_analyses = run_game(results_table = df_results, 
word_stats_table = df_word_stats)

Welcome to Type Through the Bible!
You haven't typed any Bible verses yet today.
Version 1 selected. Note that you can exit a test in progress by typing 'exit' and then hitting Enter.
Select a verse to type! Enter 0 to receive a random verse
or enter a verse number (see 'Verse_Order column of
the WEB_Catholic_Version.csv spreadsheet for a list of numbers to enter)
to select a specific verse.
You can also enter -2 to receive a random verse that you haven't yet typed
or -3 to choose the first Bible verse that hasn't yet been typed.
31137 verses have not yet been typed.
Welcome to the typing test! Your verse to type is NUM 4:10 (verse 3754 within the Bible .csv file).

Press Enter to begin the test!
Here is the verse:

They shall put it and all its vessels within a covering of sealskin, and shall put it on the frame.
Start!
Exiting typing test.
You haven't typed any Bible verses yet today.
Press 0 to retry the verse you just typed; 1 to type the next verse; 2 to type the next verse that h

In [20]:
df_results

,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms)
Test_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023.0,10.0,0.0,1.0,2023-10-24,17.0,2.0,2.0,1.0,2023-10-24 00:17:44.588771456,2023.0,10.0,2023-10-24,0.0,17.0,2.0,2.0,1.0,NaN,3944,7.549824,NaN,NaN,NaN,NaN,NaN
2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023.0,10.0,0.0,1.0,2023-10-24,18.0,2.0,2.0,1.0,2023-10-24 00:18:20.163038180,2023.0,10.0,2023-10-24,0.0,18.0,2.0,2.0,1.0,NaN,3388,20.586166,NaN,NaN,NaN,NaN,NaN
3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023.0,10.0,0.0,1.0,2023-10-24,18.0,2.0,2.0,1.0,2023-10-24 00:18:29.095581096,2023.0,10.0,2023-10-24,0.0,18.0,2.0,2.0,1.0,NaN,1464,65.697538,NaN,NaN,NaN,NaN,NaN
4,1.698121e+09,2023-10-24 00:21:24.331389,2023-10-24 04:21:24.331389+00:00,56,4.878704,11.478458,137.741497,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,124.569,2023.0,10.0,0.0,1.0,2023-10-24,21.0,3.0,2.0,1.0,2023-10-24 00:21:29.210093071,2023.0,10.0,2023-10-24,0.0,21.0,2.0,3.0,1.0,NaN,1850,56.647128,NaN,NaN,NaN,NaN,NaN
5,1.698121e+09,2023-10-24 00:22:03.749152,2023-10-24 04:22:03.749152+00:00,135,19.078445,7.076048,84.912580,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,116.638,2023.0,10.0,0.0,1.0,2023-10-24,22.0,3.0,2.0,1.0,2023-10-24 00:22:22.827597200,2023.0,10.0,2023-10-24,0.0,22.0,2.0,3.0,1.0,NaN,4206,1.406800,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,1.701996e+09,2023-12-07 19:31:41.216187,2023-12-08 00:31:41.216187+00:00,185,19.186214,9.642340,115.708082,NUM,4,3751,"""On the table of show bread they shall spread ...",7.0,132.705476,128.780425,133.934766,133.195,2023.0,12.0,19.0,1.0,2023-12-07,31.0,4.0,3.0,2.0,2023-12-07 19:32:00.402400800,2023.0,12.0,2023-12-07,19.0,32.0,3.0,4.0,2.0,0.0,3391,20.515826,10.810811,10.810811,NaN,NaN,NaN
4262,1.701996e+09,2023-12-07 19:32:01.025332,2023-12-08 00:32:01.025332+00:00,112,9.103476,12.302993,147.635916,NUM,4,3752,"They shall spread on them a scarlet cloth, and...",8.0,135.349267,128.831273,133.945498,133.198,2023.0,12.0,19.0,1.0,2023-12-07,32.0,4.0,3.0,2.0,2023-12-07 19:32:10.128807900,2023.0,12.0,2023-12-07,19.0,32.0,3.0,4.0,2.0,0.0,1089,74.490035,4.464286,4.464286,NaN,NaN,NaN
4263,1.701996e+09,2023-12-07 19:32:10.719891,2023-12-08 00:32:10.719891+00:00,168,14.232376,11.804072,141.648869,NUM,4,3753,"""They shall take a blue cloth and cover the la...",9.0,135.235827,128.737423,133.969192,133.200,2023.0,12.0,19.0,1.0,2023-12-07,32.0,4.0,3.0,2.0,2023-12-07 19:32:24.952267300,2023.0,12.0,2023-12-07,19.0,32.0,3.0,4.0,2.0,0.0,1544,63.821805,4.761905,4.761905,NaN,NaN,NaN


If df_results is blank (e.g. because the player exited out of his/her first typing test during his/her first game), some of the following code will likely crash, because they are expecting results to be present within df_results. Therefore, the program will exit out early instead of continuing on.

In [21]:
if len(df_results) == 0:
    print("No results have been entered, so there is nothing to save or \
analyze. Exiting program in 5 seconds.")
    time.sleep(5) # Allows the user to view the above message
    raise SystemExit # See https://stackoverflow.com/a/19747562/13097194

In [22]:
# Updating certain df_Bible columns to reflect new results:

In [23]:
df_Bible['Characters_Typed'] = df_Bible['Characters'] * df_Bible['Typed']
df_Bible['Total_Characters_Typed'] = df_Bible['Characters'] * df_Bible['Tests']
df_Bible

,Book_Order,Book_Name,Chapter_Name,Book_and_Chapter,Chapter_Order,Verse_#,Verse,Characters,Typed,Tests,Fastest_WPM,Characters_Typed,Total_Characters_Typed,Count
Verse_Order,,,,,,,,,,,,,,
1,1,GEN,1,GEN 1,1,1,"In the beginning, God created the heavens and ...",56,1,8,193.251509,56,448,1
2,1,GEN,1,GEN 1,1,2,The earth was formless and empty. Darkness was...,135,1,2,140.461827,135,270,1
3,1,GEN,1,GEN 1,1,3,"God said, ""Let there be light,"" and there was ...",52,1,1,84.008493,52,52,1
4,1,GEN,1,GEN 1,1,4,"God saw the light, and saw that it was good. G...",85,1,1,123.198378,85,85,1
5,1,GEN,1,GEN 1,1,5,"God called the light ""day"", and the darkness h...",119,1,1,135.664272,119,119,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35375,95,REV,22,REV 22,1328,17,"The Spirit and the bride say, ""Come!"" He who h...",160,0,0,NaN,0,0,1
35376,95,REV,22,REV 22,1328,18,I testify to everyone who hears the words of t...,159,0,0,NaN,0,0,1
35377,95,REV,22,REV 22,1328,19,If anyone takes away from the words of the boo...,174,0,0,NaN,0,0,1


In [24]:
characters_typed_sum = df_Bible['Characters_Typed'].sum()
proportion_of_Bible_typed = characters_typed_sum / df_Bible['Characters'].sum()

print(f"You have typed {characters_typed_sum} characters so far, \
which represents {round(100*proportion_of_Bible_typed, 4)}% of the Bible.")

You have typed 538841 characters so far, which represents 11.9526% of the Bible.


# Adding additional values and statistics to df_results:

(The following cell was derived from [this script](https://github.com/kburchfiel/typeracer_data_analyzer/blob/master/typeracer_data_analyzer_v2.ipynb) that I wrote.)

These statistics will get recreated whenever the script is run; this approach allows for the results to be revised as needed (e.g. if certain rows are removed from the dataset).

In [25]:
df_results['Last 10 Avg'] = df_results['WPM'].rolling(10).mean()
df_results['Last 100 Avg'] = df_results['WPM'].rolling(100).mean()
df_results['Last 1000 Avg'] = df_results['WPM'].rolling(1000).mean()


df_results['Local_Start_Year'] = df_results['Local_Start_Time'].dt.year
df_results['Local_Start_Month'] = df_results['Local_Start_Time'].dt.month
df_results['Local_Start_Date'] = df_results['Local_Start_Time'].dt.date
df_results['Local_Start_Hour'] = df_results['Local_Start_Time'].dt.hour
df_results['Local_Start_Minute'] = df_results['Local_Start_Time'].dt.minute
df_results['Local_Start_10_Minute_Block'] = df_results[
'Local_Start_Minute'] // 10 + 1
df_results['Local_Start_15_Minute_Block'] = df_results[
'Local_Start_Minute'] // 15 + 1
df_results['Local_Start_30_Minute_Block'] = df_results[
'Local_Start_Minute'] // 30 + 1

df_results['Count'] = 1 # Useful for pivot tables that analyze test counts
# by book, month, etc.

# In order to more accurately calculate the largest number of characters 
# typed within a given block of time, we'll want to know the end time
# of each test. This can be calculated as the sum of the local start time
# and the number of seconds each test took.
df_results['Local_End_Time'] = df_results[
'Local_Start_Time'] + pd.to_timedelta(df_results['Seconds'], unit = 's')

df_results['Local_End_Year'] = df_results['Local_End_Time'].dt.year
df_results['Local_End_Month'] = df_results['Local_End_Time'].dt.month
df_results['Local_End_Date'] = df_results['Local_End_Time'].dt.date
df_results['Local_End_Hour'] = df_results['Local_End_Time'].dt.hour
df_results['Local_End_Minute'] = df_results['Local_End_Time'].dt.minute
df_results['Local_End_15_Minute_Block'] = df_results[
'Local_End_Minute'] // 15 + 1
df_results['Local_End_10_Minute_Block'] = df_results[
'Local_End_Minute'] // 10 + 1
df_results['Local_End_30_Minute_Block'] = df_results[
'Local_End_Minute'] // 30 + 1

# The following line uses a list comprehension to generate a cumulative average
# of all WPM scores up until the current race. .iloc searches from 0 to i+1 for
# each row so that that row is included in the calculation.
df_results['cumulative_avg'] = [round(np.mean(df_results.iloc[0:i+1]['WPM']),
3) for i in range(len(df_results))]
df_results

,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms)
Test_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023,10,0,1,2023-10-24,17,2,2,1,2023-10-24 00:17:44.588771456,2023,10,2023-10-24,0,17,2,2,1,NaN,3944,7.549824,NaN,NaN,NaN,NaN,NaN
2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:20.163038180,2023,10,2023-10-24,0,18,2,2,1,NaN,3388,20.586166,NaN,NaN,NaN,NaN,NaN
3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:29.095581096,2023,10,2023-10-24,0,18,2,2,1,NaN,1464,65.697538,NaN,NaN,NaN,NaN,NaN
4,1.698121e+09,2023-10-24 00:21:24.331389,2023-10-24 04:21:24.331389+00:00,56,4.878704,11.478458,137.741497,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,124.569,2023,10,0,1,2023-10-24,21,3,2,1,2023-10-24 00:21:29.210093071,2023,10,2023-10-24,0,21,2,3,1,NaN,1850,56.647128,NaN,NaN,NaN,NaN,NaN
5,1.698121e+09,2023-10-24 00:22:03.749152,2023-10-24 04:22:03.749152+00:00,135,19.078445,7.076048,84.912580,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,116.638,2023,10,0,1,2023-10-24,22,3,2,1,2023-10-24 00:22:22.827597200,2023,10,2023-10-24,0,22,2,3,1,NaN,4206,1.406800,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,1.701996e+09,2023-12-07 19:31:41.216187,2023-12-08 00:31:41.216187+00:00,185,19.186214,9.642340,115.708082,NUM,4,3751,"""On the table of show bread they shall spread ...",7.0,132.705476,128.780425,133.934766,133.195,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:32:00.402400800,2023,12,2023-12-07,19,32,3,4,2,0.0,3391,20.515826,10.810811,10.810811,NaN,NaN,NaN
4262,1.701996e+09,2023-12-07 19:32:01.025332,2023-12-08 00:32:01.025332+00:00,112,9.103476,12.302993,147.635916,NUM,4,3752,"They shall spread on them a scarlet cloth, and...",8.0,135.349267,128.831273,133.945498,133.198,2023,12,19,1,2023-12-07,32,4,3,2,2023-12-07 19:32:10.128807900,2023,12,2023-12-07,19,32,3,4,2,0.0,1089,74.490035,4.464286,4.464286,NaN,NaN,NaN
4263,1.701996e+09,2023-12-07 19:32:10.719891,2023-12-08 00:32:10.719891+00:00,168,14.232376,11.804072,141.648869,NUM,4,3753,"""They shall take a blue cloth and cover the la...",9.0,135.235827,128.737423,133.969192,133.200,2023,12,19,1,2023-12-07,32,4,3,2,2023-12-07 19:32:24.952267300,2023,12,2023-12-07,19,32,3,4,2,0.0,1544,63.821805,4.761905,4.761905,NaN,NaN,NaN


# Adding additional metrics to df_word_stats:

In [26]:
df_word_stats['Count'] = 1
df_word_stats['characters'] = df_word_stats['word'].str.len()
df_word_stats['CPS'] = df_word_stats['characters'] / (df_word_stats['word_duration (ms)'] / 1000)
df_word_stats['WPM'] = df_word_stats['CPS'] * 12
df_word_stats

,word,typed_word_without_mistakes,word_duration (ms),Count,characters,CPS,WPM
0,In,1,261.6661,1,2,7.643329,91.719944
1,all,1,329.9691,1,3,9.091760,109.101125
2,the,1,158.9050,1,3,18.879205,226.550455
3,land,1,234.7889,1,4,17.036581,204.438966
4,of,1,133.5537,1,2,14.975250,179.702996
...,...,...,...,...,...,...,...
9466,which,1,327.8958,1,5,15.248747,182.984960
9467,they,1,231.6682,1,4,17.266073,207.192873
9468,minister,1,553.3430,1,8,14.457579,173.490945
9469,to,1,183.4473,1,2,10.902314,130.827764


In [27]:
print("Saving results:")

Saving results:


In [28]:
def attempt_save(df, filename, index):
    '''This function attempts to save the DataFrame passed to df to the file
    specified by filename. It allows players to retry the save operation
    if it wasn't initially successful (e.g. because the file was open at 
    the time), thus preventing them from losing their latest progress.
    The index parameter determines whether or not the DataFrame's index
    will be included in the .csv export. Set to True for results.csv
    but False for Web_Catholic_Version_for_game_updated.csv.'''
    while True:
        try: 
            df.to_csv(filename, index = index)
            return
        except:
            print("File could not be saved, likely because it is currently open. \
Try closing the file and trying again. Press Enter to retry.")
            input()

In [29]:
attempt_save(df_Bible, 'WEB_Catholic_Version_for_game_updated.csv', index = True)
# The verse_order index will be stored within this .csv file so that it can
# be accessed during the subsequent run.

In [30]:
attempt_save(df_results, 'results.csv', index = True)

In [31]:
attempt_save(df_word_stats, 'word_stats.csv', index = False)

In [32]:
print("Successfully saved updated copies of the Results, Word Stats, \
and Bible .csv files.")

Successfully saved updated copies of the Results, Word Stats, and Bible .csv files.


In [33]:
if run_analyses == 0: # In this case, the analysis portion of the script will
    # be skipped. These analyses can be updated when the game is next played.
    print("Analyses won't be updated during this session. \
Exiting program in 3 seconds.")
    time.sleep(3) # Allows the user to view the above message
    raise SystemExit # See https://stackoverflow.com/a/19747562/13097194

# Visualizing the player's progress in typing the entire Bible:

In [34]:
analysis_start_time = time.time() # Allows us to determine how long the
# analyses took
print("Updating analyses:")

Updating analyses:


In [35]:
df_Bible['Count'] = 1

### Creating a tree map within Plotly that visualizes the player's progress in typing the entire Bible:

In [ ]:
print("Creating tree map(s).")

In [36]:
# This code is based on https://plotly.com/python/treemaps/
# It's pretty amazing that such a complex visualization can be created using
# just one line of code. Thanks Plotly!
fig_tree_map_books_chapters_verses = px.treemap(
    df_Bible, path = ['Book_Name', 'Chapter_Name', 'Verse_#'], 
    values = 'Characters', color = 'Typed')
# fig_verses_typed

In [37]:
fig_tree_map_books_chapters_verses.write_html(
    'Analyses/tree_map_books_chapters_verses.html')

In [38]:
# # A similar chart that doesn't use the Typed column for color coding:
# (This chart, unlike fig_verses_typed above, won't change unless edits are 
# made to the code itself, so it can be 
# commented out after being run once.)
# fig_Bible_verses.write_html('Bible_tree_map.html')
# fig_Bible_verses = px.treemap(df_Bible, path = ['Book_Name', 
# 'Chapter_Name', 'Verse_#'], values = 'Characters')
# fig_Bible_verses

In [39]:
df_Bible

,Book_Order,Book_Name,Chapter_Name,Book_and_Chapter,Chapter_Order,Verse_#,Verse,Characters,Typed,Tests,Fastest_WPM,Characters_Typed,Total_Characters_Typed,Count
Verse_Order,,,,,,,,,,,,,,
1,1,GEN,1,GEN 1,1,1,"In the beginning, God created the heavens and ...",56,1,8,193.251509,56,448,1
2,1,GEN,1,GEN 1,1,2,The earth was formless and empty. Darkness was...,135,1,2,140.461827,135,270,1
3,1,GEN,1,GEN 1,1,3,"God said, ""Let there be light,"" and there was ...",52,1,1,84.008493,52,52,1
4,1,GEN,1,GEN 1,1,4,"God saw the light, and saw that it was good. G...",85,1,1,123.198378,85,85,1
5,1,GEN,1,GEN 1,1,5,"God called the light ""day"", and the darkness h...",119,1,1,135.664272,119,119,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35375,95,REV,22,REV 22,1328,17,"The Spirit and the bride say, ""Come!"" He who h...",160,0,0,NaN,0,0,1
35376,95,REV,22,REV 22,1328,18,I testify to everyone who hears the words of t...,159,0,0,NaN,0,0,1
35377,95,REV,22,REV 22,1328,19,If anyone takes away from the words of the boo...,174,0,0,NaN,0,0,1


In [40]:
# This variant of the treemap shows chapters and verses rather than books,
# chapters, and verses.
if (run_on_notebook == True) & (extra_analyses == True):
    fig_tree_map_chapters_verses = px.treemap(df_Bible, path = [
        'Book_and_Chapter', 'Verse_#'], values = 'Characters', color = 'Typed')
    fig_tree_map_chapters_verses.write_html(
        'Analyses/tree_map_chapters_verses.html')
    fig_tree_map_chapters_verses.write_image(
        'Analyses/tree_map_chapters_verses.png', width = 7680, height = 4320)

In [96]:
extra_analyses = True

In [97]:
# This variant of the treemap shows each verse as its own box, which results in 
# a very busy graph that takes a while to load within a web browser
# (if it even loads at all).

if (run_on_notebook == True) & (extra_analyses == True):
    fig_tree_map_verses = px.treemap(df_Bible, path = [df_Bible.index], 
                                     values = 'Characters', color = 'Typed')
    fig_tree_map_verses.write_html('Analyses/tree_map_verses.html')
    fig_tree_map_verses.write_image('Analyses/tree_map_verses_8K.png', 
                                    width = 7680, height = 4320) 
    fig_tree_map_verses.write_image('Analyses/tree_map_verses_16K.png', 
                                    width = 15360, height = 8640) 
# fig_tree_map_verses.write_image('Analyses/tree_map_verses.png', width = 30720, 
# height = 17280) # Didn't end up rendering successfully, probably 
# because the dimensions were absurdly large!

### Creating a bar chart that shows the proportion of each book that has been typed so far:

In [ ]:
print("Creating progress analyses.")

In [42]:
df_characters_typed_by_book = df_Bible.pivot_table(index = ['Book_Order', 
'Book_Name'], values = ['Characters', 'Characters_Typed'], 
aggfunc = 'sum').reset_index()
# Adding 'Book_Order' as the first index value allows for the pivot tables
# and bars to be ordered by that value.
df_characters_typed_by_book['proportion_typed'] = df_characters_typed_by_book[
    'Characters_Typed'] / df_characters_typed_by_book['Characters']
df_characters_typed_by_book.to_csv(
    'Analyses/characters_typed_by_book.csv')
df_characters_typed_by_book

,Book_Order,Book_Name,Characters,Characters_Typed,proportion_typed
0,1,GEN,185293,185293,1.000000
1,2,EXO,159492,159492,1.000000
2,3,LEV,120367,120367,1.000000
3,4,NUM,165525,18864,0.113965
4,5,DEU,139344,0,0.000000
...,...,...,...,...,...
68,91,1JN,12363,12363,1.000000
69,92,2JN,1536,453,0.294922
70,93,3JN,1525,1525,1.000000
71,94,JUD,3461,136,0.039295


In [43]:
fig_proportion_of_each_book_typed = px.bar(df_characters_typed_by_book, 
x = 'Book_Name', y = 'proportion_typed')
fig_proportion_of_each_book_typed.update_yaxes(range = [0, 1]) # Setting
# the maximum y value as 1 better demonstrates how much of the Bible
# has been typed so far
fig_proportion_of_each_book_typed.write_html(
    'Analyses/proportion_of_each_book_typed.html')
fig_proportion_of_each_book_typed.write_image(
    'Analyses/proportion_of_each_book_typed.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_proportion_of_each_book_typed

### Creating a chart that compares the number of characters in each book with the number that have been typed:

This provides a clearer view of the player's progress in typing the Bible, as each bar's height is based on the number of characters. (In contrast, bars for fully typed small books will be just as high in fig_proportion_of_each_book_typed as those for fully typed large books.)

In [44]:
fig_characters_typed_in_each_book = px.bar(df_characters_typed_by_book, 
x = 'Book_Name', y = ['Characters', 'Characters_Typed'], barmode = 'overlay')
fig_characters_typed_in_each_book.write_html(
    'Analyses/characters_typed_by_book.html')
fig_characters_typed_in_each_book.write_image(
    'Analyses/characters_typed_by_book.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_characters_typed_in_each_book

## Creating charts that show both book- and chapter-level data:

In [45]:
df_characters_typed_by_book_and_chapter = df_Bible.pivot_table(index = [
'Book_Order', 'Book_Name', 'Book_and_Chapter'], values = [
    'Characters', 'Characters_Typed'], aggfunc = 'sum').reset_index()
df_characters_typed_by_book_and_chapter[
'proportion_typed'] = df_characters_typed_by_book_and_chapter[
'Characters_Typed'] / df_characters_typed_by_book_and_chapter['Characters']
df_characters_typed_by_book_and_chapter.to_csv(
    'Analyses/characters_typed_by_book_and_chapter.csv')
df_characters_typed_by_book_and_chapter

,Book_Order,Book_Name,Book_and_Chapter,Characters,Characters_Typed,proportion_typed
0,1,GEN,GEN 1,3831,3831,1.0
1,1,GEN,GEN 10,2582,2582,1.0
2,1,GEN,GEN 11,3435,3435,1.0
3,1,GEN,GEN 12,2473,2473,1.0
4,1,GEN,GEN 13,2169,2169,1.0
...,...,...,...,...,...,...
1323,95,REV,REV 5,2054,0,0.0
1324,95,REV,REV 6,2503,0,0.0
1325,95,REV,REV 7,2452,0,0.0
1326,95,REV,REV 8,1900,0,0.0


The following chart shows both books (as bars) and chapters (as sections of these bars). These sections are also color coded by the proportion of each chapter that has been typed.

In [46]:
fig_characters_typed_in_each_book_and_chapter = px.bar(
df_characters_typed_by_book_and_chapter, x = 'Book_Name', y = [
    'Characters'], color = 'proportion_typed')
fig_characters_typed_in_each_book_and_chapter.write_html(
    'Analyses/characters_typed_by_book_and_chapter.html')
fig_characters_typed_in_each_book_and_chapter.write_image(
    'Analyses/characters_typed_by_book_and_chapter.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_characters_typed_in_each_book_and_chapter

## Creating similar charts at the chapter level:

These proved difficult to interpret due to the narrowness of the bars, so I'm commenting this code out for now.

In [47]:
# fig_proportion_of_each_chapter_typed = px.bar(df_characters_typed_by_chapter, 
# x = 'Book_and_Chapter', y = 'proportion_typed')
# fig_proportion_of_each_chapter_typed.update_yaxes(range = [0, 1]) # Setting
# # the maximum y value as 1 better demonstrates how much of the Bible
# # has been typed so far
# fig_proportion_of_each_chapter_typed.write_html(
# 'Analyses/proportion_of_each_chapter_typed.html')
# fig_proportion_of_each_chapter_typed

# fig_characters_typed_in_each_chapter = px.bar(df_characters_typed_by_chapter, 
# x = 'Book_and_Chapter', y = ['Characters', 'Characters_Typed'], 
# barmode = 'overlay')
# fig_characters_typed_in_each_chapter.write_html(
# 'Analyses/characters_typed_by_chapter.html')
# fig_characters_typed_in_each_chapter

# Endurance statistics (e.g. most characters/verses typed over a given time period):

In [ ]:
print("Analyzing typing activity by period.")

## Calculating the dates with the most characters and verses typed:

Note: In order to create more accurate analyses, I will filter the results to only include values with the same start and end periods. (For instance, if a given test began at 9:59 p.m. on 2023-11-17 but ended after 10 p.m., that test would get filtered out of a 'top hours by characters typed' report, since including it would extend the time frame analyzed beyond a 60-minute window.)

In [48]:
df_top_dates_by_characters = df_results.pivot_table(
    index = ['Local_Start_Date', 'Local_End_Date'], values = 'Characters', aggfunc = 'sum').reset_index(
    ).sort_values('Characters', ascending = False)
# By using both the start and end dates as pivot index values, we've already 
# separated results with different start and end dates from ones whose 
# start and end dates are the same. (This will prevent the tests included
# in a given date's calculation from extending beyond just that date.)
# We'll also filter the DataFrame to exclude any results whose start
# and end dates differ:
df_top_dates_by_characters = df_top_dates_by_characters.query("Local_Start_Date == Local_End_Date").head(50).copy()
df_top_dates_by_characters['Rank'] = df_top_dates_by_characters[
    'Characters'].rank(ascending = False, method = 'min').astype('int')
# Creating a column that shows both the rank and date: (This also prevents
# Plotly from converting the x axis to a date range, which would interfere
# with the order of the chart items)
df_top_dates_by_characters['Rank and Date'] = '#'+df_top_dates_by_characters[
    'Rank'].astype('str') + ': ' + df_top_dates_by_characters[
        'Local_Start_Date'].astype('str')
df_top_dates_by_characters.reset_index(drop=True,inplace=True)
df_top_dates_by_characters.to_csv(
    'Analyses/top_dates_by_characters.csv', index = False)
df_top_dates_by_characters

,Local_Start_Date,Local_End_Date,Characters,Rank,Rank and Date
0,2023-11-19,2023-11-19,52402,1,#1: 2023-11-19
1,2023-11-18,2023-11-18,48124,2,#2: 2023-11-18
2,2023-12-02,2023-12-02,41568,3,#3: 2023-12-02
3,2023-11-28,2023-11-28,33352,4,#4: 2023-11-28
4,2023-11-12,2023-11-12,31286,5,#5: 2023-11-12
5,2023-10-29,2023-10-29,29415,6,#6: 2023-10-29
6,2023-11-05,2023-11-05,25289,7,#7: 2023-11-05
7,2023-11-26,2023-11-26,25110,8,#8: 2023-11-26
8,2023-12-07,2023-12-07,24919,9,#9: 2023-12-07
9,2023-12-01,2023-12-01,21786,10,#10: 2023-12-01


In [49]:
fig_top_dates_by_characters = px.bar(df_top_dates_by_characters, 
x = 'Rank and Date', y = 'Characters', text = 'Characters')
fig_top_dates_by_characters.update_xaxes(tickangle = 90)
fig_top_dates_by_characters.write_html('Analyses/top_dates_by_characters.html')
fig_top_dates_by_characters.write_image(
    'Analyses/top_dates_by_characters.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_dates_by_characters

In [50]:
df_top_dates_by_verses = df_results.pivot_table(
    index = ['Local_Start_Date', 'Local_End_Date'], 
    values = 'Count', aggfunc = 'sum').reset_index(
    ).rename(columns = {'Count':'Verses'}).sort_values(
        'Verses', ascending = False)
df_top_dates_by_verses = df_top_dates_by_verses.query(
    "Local_Start_Date == Local_End_Date").head(50).copy()
df_top_dates_by_verses['Rank'] = df_top_dates_by_verses['Verses'].rank(
    ascending = False, method = 'min').astype('int')
df_top_dates_by_verses['Rank and Date'] = '#'+df_top_dates_by_verses[
    'Rank'].astype('str') + ': ' + df_top_dates_by_verses[
        'Local_Start_Date'].astype('str')
df_top_dates_by_verses.reset_index(drop=True,inplace=True)
df_top_dates_by_verses.to_csv('Analyses/top_dates_by_verses.csv', index = False)
df_top_dates_by_verses

,Local_Start_Date,Local_End_Date,Verses,Rank,Rank and Date
0,2023-11-19,2023-11-19,379,1,#1: 2023-11-19
1,2023-11-18,2023-11-18,369,2,#2: 2023-11-18
2,2023-12-02,2023-12-02,310,3,#3: 2023-12-02
3,2023-11-12,2023-11-12,261,4,#4: 2023-11-12
4,2023-10-29,2023-10-29,251,5,#5: 2023-10-29
5,2023-11-28,2023-11-28,244,6,#6: 2023-11-28
6,2023-12-07,2023-12-07,203,7,#7: 2023-12-07
7,2023-11-05,2023-11-05,201,8,#8: 2023-11-05
8,2023-11-26,2023-11-26,195,9,#9: 2023-11-26
9,2023-12-01,2023-12-01,171,10,#10: 2023-12-01


In [51]:
fig_top_dates_by_verses = px.bar(df_top_dates_by_verses, 
x = 'Rank and Date', y = 'Verses', text = 'Verses')
fig_top_dates_by_verses.update_xaxes(tickangle = 90)
fig_top_dates_by_verses.write_html('Analyses/top_dates_by_verses.html')
fig_top_dates_by_verses.write_image(
    'Analyses/top_dates_by_verses.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_dates_by_verses

## Performing similar analyses by month:

In [52]:
df_top_months_by_characters = df_results.pivot_table(
    index = ['Local_Start_Year', 'Local_End_Year', 
    'Local_Start_Month', 'Local_End_Month'], 
    values = 'Characters', aggfunc = 'sum').reset_index(
    ).sort_values('Characters', ascending = False)
df_top_months_by_characters = df_top_months_by_characters.query(
    "Local_Start_Year == Local_End_Year & \
Local_Start_Month == Local_End_Month").head(50).copy()

df_top_months_by_characters['Rank'] = df_top_months_by_characters[
'Characters'].rank(ascending = False, method = 'min').astype('int')
df_top_months_by_characters['Rank and Month'] = '#'+df_top_months_by_characters[
    'Rank'].astype('str') + ': ' + df_top_months_by_characters[
        'Local_Start_Year'].astype('str') + '-' + df_top_months_by_characters[
            'Local_Start_Month'].astype('str')
df_top_months_by_characters.reset_index(drop=True,inplace=True)
df_top_months_by_characters.to_csv('Analyses/top_months_by_characters.csv', index = False)
df_top_months_by_characters

,Local_Start_Year,Local_End_Year,Local_Start_Month,Local_End_Month,Characters,Rank,Rank and Month
0,2023,2023,11,11,365316,1,#1: 2023-11
1,2023,2023,12,12,137116,2,#2: 2023-12
2,2023,2023,10,10,38076,3,#3: 2023-10


In [53]:
fig_top_months_by_characters = px.bar(df_top_months_by_characters, 
x = 'Rank and Month', y = 'Characters', text = 'Characters')
fig_top_months_by_characters.update_xaxes(tickangle = 90)
fig_top_months_by_characters.write_html(
    'Analyses/top_months_by_characters.html')
fig_top_months_by_characters.write_image(
    'Analyses/top_months_by_characters.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_months_by_characters

In [54]:
df_top_months_by_verses = df_results.pivot_table(
    index = ['Local_Start_Year', 'Local_Start_Month', 
    'Local_End_Year', 'Local_End_Month'], 
    values = 'Count', aggfunc = 'sum').reset_index(
    ).rename(columns={'Count':'Verses'}).sort_values(
        'Verses', ascending = False)
df_top_months_by_verses = df_top_months_by_verses.query(
    "Local_Start_Year == Local_End_Year & \
Local_Start_Month == Local_End_Month").head(50).copy()

df_top_months_by_verses['Rank'] = df_top_months_by_verses['Verses'].rank(
    ascending = False, method = 'min').astype('int')
df_top_months_by_verses['Rank and Month'] = '#'+df_top_months_by_verses[
    'Rank'].astype('str') + ': ' + df_top_months_by_verses[
        'Local_Start_Year'].astype('str') + '-' + df_top_months_by_verses[
            'Local_Start_Month'].astype('str')
df_top_months_by_verses.reset_index(drop=True,inplace=True)
df_top_months_by_verses.to_csv('Analyses/top_months_by_verses.csv', index = False)
df_top_months_by_verses

,Local_Start_Year,Local_Start_Month,Local_End_Year,Local_End_Month,Verses,Rank,Rank and Month
0,2023,11,2023,11,2863,1,#1: 2023-11
1,2023,12,2023,12,1070,2,#2: 2023-12
2,2023,10,2023,10,332,3,#3: 2023-10


In [55]:
fig_top_months_by_verses = px.bar(df_top_months_by_verses, 
x = 'Rank and Month', y = 'Verses', text = 'Verses')
fig_top_months_by_verses.update_xaxes(tickangle = 90)
fig_top_months_by_verses.write_html('Analyses/top_months_by_verses.html')
fig_top_months_by_verses.write_image(
    'Analyses/top_months_by_verses.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_months_by_verses

## Performing similar analyses for hours and for 30-, 15-, and 10-minute blocks:

In [56]:
df_top_hours_by_characters = df_results.pivot_table(index = ['Local_Start_Date', 
'Local_End_Date', 'Local_Start_Hour', 'Local_End_Hour'], values = 'Characters', 
aggfunc = 'sum').reset_index().sort_values('Characters', ascending = False)
df_top_hours_by_characters = df_top_hours_by_characters.query(
"Local_Start_Date == Local_End_Date & \
Local_Start_Hour == Local_End_Hour").head(100).copy()
df_top_hours_by_characters['Hour'] = df_top_hours_by_characters[
'Local_Start_Date'].astype('str') + ' ' + df_top_hours_by_characters[
'Local_Start_Hour'].astype('str') 
df_top_hours_by_characters.to_csv('Analyses/top_hours_by_characters.csv', index = False)
df_top_hours_by_characters

,Local_Start_Date,Local_End_Date,Local_Start_Hour,Local_End_Hour,Characters,Hour
44,2023-11-18,2023-11-18,13,13,32183,2023-11-18 13
27,2023-11-12,2023-11-12,15,15,28510,2023-11-12 15
52,2023-11-19,2023-11-19,17,17,27405,2023-11-19 17
10,2023-10-29,2023-10-29,19,19,21991,2023-10-29 19
74,2023-11-26,2023-11-26,0,0,18860,2023-11-26 0
...,...,...,...,...,...,...
62,2023-11-21,2023-11-21,0,0,137,2023-11-21 0
67,2023-11-24,2023-11-24,21,21,124,2023-11-24 21
98,2023-12-04,2023-12-04,23,23,104,2023-12-04 23
13,2023-11-01,2023-11-01,22,22,103,2023-11-01 22


In [57]:
fig_top_hours_by_characters = px.bar(df_top_hours_by_characters, 
x = 'Hour', y = 'Characters', text = 'Characters')
fig_top_hours_by_characters.update_xaxes(type = 'category')
fig_top_hours_by_characters.write_html('Analyses/top_hours_by_characters.html')
fig_top_hours_by_characters.write_image(
    'Analyses/top_hours_by_characters.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_hours_by_characters

In [58]:
df_top_30m_by_characters = df_results.pivot_table(index = ['Local_Start_Date', 
'Local_End_Date', 'Local_Start_Hour', 'Local_End_Hour', 
'Local_Start_30_Minute_Block', 'Local_End_30_Minute_Block'], 
values = 'Characters', aggfunc = 'sum').reset_index().sort_values(
'Characters', ascending = False)
df_top_30m_by_characters = df_top_30m_by_characters.query(
"Local_Start_Date == Local_End_Date & Local_Start_Hour == Local_End_Hour \
& Local_Start_30_Minute_Block == Local_End_30_Minute_Block").head(100).copy()
df_top_30m_by_characters['30-Minute Block'] = df_top_30m_by_characters[
'Local_Start_Date'].astype('str') + ' ' + df_top_30m_by_characters[
'Local_Start_Hour'].astype('str') + '_' + df_top_30m_by_characters[
'Local_Start_30_Minute_Block'].astype('str')
df_top_30m_by_characters.to_csv('Analyses/top_30m_by_characters.csv', index = False)
df_top_30m_by_characters

,Local_Start_Date,Local_End_Date,Local_Start_Hour,Local_End_Hour,Local_Start_30_Minute_Block,Local_End_30_Minute_Block,Characters,30-Minute Block
70,2023-11-19,2023-11-19,17,17,2,2,16558,2023-11-19 17_2
59,2023-11-18,2023-11-18,13,13,2,2,16239,2023-11-18 13_2
57,2023-11-18,2023-11-18,13,13,1,1,15880,2023-11-18 13_1
34,2023-11-12,2023-11-12,15,15,1,1,14607,2023-11-12 15_1
36,2023-11-12,2023-11-12,15,15,2,2,13802,2023-11-12 15_2
...,...,...,...,...,...,...,...,...
80,2023-11-20,2023-11-20,21,21,1,1,627,2023-11-20 21_1
18,2023-11-02,2023-11-02,0,0,1,1,612,2023-11-02 0_1
7,2023-10-28,2023-10-28,0,0,2,2,549,2023-10-28 0_2
46,2023-11-16,2023-11-16,18,18,2,2,502,2023-11-16 18_2


In [59]:
fig_top_30m_by_characters = px.bar(df_top_30m_by_characters, 
x = '30-Minute Block', y = 'Characters', text = 'Characters')
fig_top_30m_by_characters.update_xaxes(type = 'category')
fig_top_30m_by_characters.write_html(
'Analyses/top_30m_blocks_by_characters.html')
fig_top_30m_by_characters.write_image(
    'Analyses/top_30m_blocks_by_characters.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_30m_by_characters

In [60]:
df_top_15m_by_characters = df_results.pivot_table(index = [
'Local_Start_Date', 'Local_End_Date', 'Local_Start_Hour', 'Local_End_Hour', 
'Local_Start_15_Minute_Block', 'Local_End_15_Minute_Block'], 
values = 'Characters', aggfunc = 'sum').reset_index().sort_values(
'Characters', ascending = False)
# print(len(df_top_15m_by_characters))
df_top_15m_by_characters = df_top_15m_by_characters.query(
"Local_Start_Date == Local_End_Date & Local_Start_Hour == Local_End_Hour & \
Local_Start_15_Minute_Block == Local_End_15_Minute_Block").head(100).copy()
df_top_15m_by_characters['15-Minute Block'] = df_top_15m_by_characters[
'Local_Start_Date'].astype('str') + ' ' + df_top_15m_by_characters[
'Local_Start_Hour'].astype('str') + '_' + df_top_15m_by_characters[
'Local_Start_15_Minute_Block'].astype('str')
# print(len(df_top_15m_by_characters))
df_top_15m_by_characters.to_csv('Analyses/top_15m_by_characters.csv', index = False)
df_top_15m_by_characters

,Local_Start_Date,Local_End_Date,Local_Start_Hour,Local_End_Hour,Local_Start_15_Minute_Block,Local_End_15_Minute_Block,Characters,15-Minute Block
184,2023-12-01,2023-12-01,9,9,1,1,8738,2023-12-01 9_1
105,2023-11-19,2023-11-19,17,17,2,2,8654,2023-11-19 17_2
198,2023-12-02,2023-12-02,21,21,2,2,8364,2023-12-02 21_2
226,2023-12-07,2023-12-07,9,9,2,2,8358,2023-12-07 9_2
109,2023-11-19,2023-11-19,17,17,4,4,8299,2023-11-19 17_4
...,...,...,...,...,...,...,...,...
103,2023-11-19,2023-11-19,17,17,1,1,1792,2023-11-19 17_1
131,2023-11-21,2023-11-21,19,19,1,1,1790,2023-11-21 19_1
151,2023-11-25,2023-11-25,23,23,3,3,1765,2023-11-25 23_3
101,2023-11-19,2023-11-19,10,10,1,1,1744,2023-11-19 10_1


In [61]:
fig_top_15m_by_characters = px.bar(df_top_15m_by_characters, 
x = '15-Minute Block', y = 'Characters', text = 'Characters')
fig_top_15m_by_characters.update_xaxes(type = 'category')
fig_top_15m_by_characters.write_html(
'Analyses/top_15m_blocks_by_characters.html')
fig_top_15m_by_characters.write_image(
    'Analyses/top_15m_blocks_by_characters.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_15m_by_characters

In [62]:
df_top_10m_by_characters = df_results.pivot_table(index = ['Local_Start_Date', 
'Local_End_Date', 'Local_Start_Hour', 'Local_End_Hour', 
'Local_Start_10_Minute_Block', 'Local_End_10_Minute_Block'], 
values = 'Characters', aggfunc = 'sum').reset_index().sort_values(
'Characters', ascending = False)
# print(len(df_top_10m_by_characters))
df_top_10m_by_characters = df_top_10m_by_characters.query(
"Local_Start_Date == Local_End_Date & Local_Start_Hour == Local_End_Hour \
& Local_Start_10_Minute_Block == Local_End_10_Minute_Block").head(100).copy()
df_top_10m_by_characters['10-Minute Block'] = df_top_10m_by_characters[
'Local_Start_Date'].astype('str') + ' ' + df_top_10m_by_characters[
'Local_Start_Hour'].astype('str') + '_' + df_top_10m_by_characters[
'Local_Start_10_Minute_Block'].astype('str')
# print(len(df_top_10m_by_characters))
df_top_10m_by_characters.to_csv('Analyses/top_10m_by_characters.csv', index = False)
df_top_10m_by_characters

,Local_Start_Date,Local_End_Date,Local_Start_Hour,Local_End_Hour,Local_Start_10_Minute_Block,Local_End_10_Minute_Block,Characters,10-Minute Block
246,2023-12-01,2023-12-01,9,9,1,1,5970,2023-12-01 9_1
147,2023-11-19,2023-11-19,17,17,3,3,5792,2023-11-19 17_3
248,2023-12-01,2023-12-01,9,9,2,2,5639,2023-12-01 9_2
261,2023-12-02,2023-12-02,18,18,2,2,5607,2023-12-02 18_2
137,2023-11-18,2023-11-18,23,23,6,6,5601,2023-11-18 23_6
...,...,...,...,...,...,...,...,...
83,2023-11-12,2023-11-12,16,16,1,1,2645,2023-11-12 16_1
165,2023-11-20,2023-11-20,16,16,3,3,2609,2023-11-20 16_3
263,2023-12-02,2023-12-02,18,18,3,3,2559,2023-12-02 18_3
157,2023-11-19,2023-11-19,18,18,2,2,2497,2023-11-19 18_2


In [63]:
fig_top_10m_by_characters = px.bar(df_top_10m_by_characters, 
x = '10-Minute Block', y = 'Characters', text = 'Characters')
fig_top_10m_by_characters.update_xaxes(type = 'category')
fig_top_10m_by_characters.write_html(
'Analyses/top_10m_blocks_by_characters.html')
fig_top_10m_by_characters.write_image(
    'Analyses/top_10m_blocks_by_characters.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_10m_by_characters

# Analyzing WPM and accuracy data:

(Some of this section's code derives from my work in [this script](https://github.com/kburchfiel/typeracer_data_analyzer/blob/master/typeracer_data_analyzer_v2.ipynb).)


In [ ]:
print("Creating WPM and accuracy analyses.")

Verses with the highest WPMs:

In [64]:
df_top_100_wpm = df_results.sort_values('WPM', ascending = False).head(
    100).copy()
df_top_100_wpm.insert(0, 'Rank', df_top_100_wpm['WPM'].rank(
    ascending = False, method = 'min').astype('int'))
# method = 'min' assigns the lowest rank to any rows that happen to have
# the same WPM. See 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html
df_top_100_wpm.to_csv('Analyses/top_100_wpm.csv')
df_top_100_wpm

,Rank,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms)
Test_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2763,1,1.701046e+09,2023-11-26 19:41:20.287025,2023-11-27 00:41:20.287025+00:00,73,4.364863,16.724465,200.693577,EXO,37,2610,He put the poles into the rings on the sides o...,5.0,144.338008,132.807831,132.639214,131.686,2023,11,19,1,2023-11-26,41,5,3,2,2023-11-26 19:41:24.651888157,2023,11,2023-11-26,19,41,3,5,2,1.0,1,100.000000,0.0,0.0,12.529850,57.394743,132.124662
29,2,1.698284e+09,2023-10-25 21:29:51.817669,2023-10-26 01:29:51.817669+00:00,56,3.477334,16.104292,193.251509,GEN,1,1,"In the beginning, God created the heavens and ...",1.0,148.772233,NaN,NaN,129.881,2023,10,21,1,2023-10-25,29,3,2,1,2023-10-25 21:29:55.295002784,2023,10,2023-10-25,21,29,2,3,1,NaN,2,99.976553,NaN,NaN,NaN,NaN,NaN
969,3,1.699821e+09,2023-11-12 15:38:07.686799,2023-11-12 20:38:07.686799+00:00,67,4.190650,15.987972,191.855668,GEN,29,818,Laban gathered together all the men of the pla...,22.0,140.791382,131.081327,NaN,132.396,2023,11,15,1,2023-11-12,38,4,3,2,2023-11-12 15:38:11.877449225,2023,11,2023-11-12,15,38,3,4,2,NaN,3,99.953107,NaN,NaN,NaN,NaN,NaN
2457,4,1.700628e+09,2023-11-21 23:35:33.409632,2023-11-22 04:35:33.409632+00:00,122,7.710312,15.822965,189.875582,EXO,28,2304,"Six of their names on the one stone, and the n...",10.0,155.285176,141.720888,130.695601,131.328,2023,11,23,1,2023-11-21,35,4,3,2,2023-11-21 23:35:41.119944128,2023,11,2023-11-21,23,35,3,4,2,1.0,4,99.929660,0.0,0.0,11.629343,57.242155,161.164761
2865,5,1.701176e+09,2023-11-28 07:47:08.693553,2023-11-28 12:47:08.693553+00:00,129,8.199585,15.732504,188.790049,EXO,40,2712,"You shall bring in the table, and set in order...",4.0,145.171311,139.548880,133.840416,131.973,2023,11,7,1,2023-11-28,47,5,4,2,2023-11-28 07:47:16.893137723,2023,11,2023-11-28,7,47,4,5,2,1.0,5,99.906213,0.0,0.0,6.971598,60.873747,160.340786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,96,1.699067e+09,2023-11-03 22:55:15.335816,2023-11-04 02:55:15.335816+00:00,92,6.374657,14.432150,173.185797,GEN,9,219,"I set my rainbow in the cloud, and it will be ...",13.0,147.654034,138.335041,NaN,136.601,2023,11,22,1,2023-11-03,55,6,4,2,2023-11-03 22:55:21.710472677,2023,11,2023-11-03,22,55,4,6,2,NaN,96,97.772567,NaN,NaN,NaN,NaN,NaN
2475,97,1.700628e+09,2023-11-21 23:41:18.702777,2023-11-22 04:41:18.702777+00:00,208,14.444523,14.399922,172.799065,EXO,28,2322,They shall bind the breastplate by its rings t...,28.0,137.195946,141.779199,130.795561,131.383,2023,11,23,1,2023-11-21,41,5,3,2,2023-11-21 23:41:33.147299620,2023,11,2023-11-21,23,41,3,5,2,1.0,97,97.749121,0.0,0.0,0.965118,59.017420,173.700333
3840,98,1.701748e+09,2023-12-04 22:48:32.099474,2023-12-05 03:48:32.099474+00:00,76,5.278625,14.397690,172.772279,LUK,4,29373,They were all filled with wrath in the synagog...,28.0,139.335278,132.075298,136.857398,133.207,2023,12,22,1,2023-12-04,48,5,4,2,2023-12-04 22:48:37.378098600,2023,12,2023-12-04,22,48,4,5,2,1.0,98,97.725674,0.0,0.0,NaN,58.450500,NaN


In [65]:
fig_top_100_wpm = px.bar(df_top_100_wpm, x = 'Rank', y = 'WPM', 
text_auto = '.6s')
fig_top_100_wpm.write_html('Analyses/top_100_wpm.html')
fig_top_100_wpm.write_image('Analyses/top_100_wpm.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_top_100_wpm

Top 'Last 10 Average' values:

In [66]:
if len(df_results) >= 10: # If fewer than 10 tests are present in df_results,
    # there won't be anything to graph (and the following code will raise
    # an error), so this section should be skipped.
    df_top_last_10_avg_results = df_results.sort_values(
    'Last 10 Avg', ascending = False).head(20).copy()
    df_top_last_10_avg_results.insert(0, 'Rank', 
    df_top_last_10_avg_results['Last 10 Avg'].rank(ascending = False, 
    method = 'min').astype('int'))
    df_top_last_10_avg_results.to_csv('Analyses/top_last_10_avg_results.csv', index = False)
    df_top_last_10_avg_results

In [67]:
if len(df_results) >= 10:
    fig_top_last_10_average_wpm = px.bar(df_top_last_10_avg_results, x = 'Rank', 
    y = 'Last 10 Avg', text_auto = '.6s')
    fig_top_last_10_average_wpm.write_html('Analyses/top_last_10_average_wpm.html')
    fig_top_last_10_average_wpm.write_image('Analyses/top_last_10_average_wpm.png', 
    width = 1920, height = 1080, engine = 'kaleido', scale = 2)
    fig_top_last_10_average_wpm

# Showing WPM results and moving averages by test number:

In [68]:
df_results

,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms)
Test_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023,10,0,1,2023-10-24,17,2,2,1,2023-10-24 00:17:44.588771456,2023,10,2023-10-24,0,17,2,2,1,NaN,3944,7.549824,NaN,NaN,NaN,NaN,NaN
2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:20.163038180,2023,10,2023-10-24,0,18,2,2,1,NaN,3388,20.586166,NaN,NaN,NaN,NaN,NaN
3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:29.095581096,2023,10,2023-10-24,0,18,2,2,1,NaN,1464,65.697538,NaN,NaN,NaN,NaN,NaN
4,1.698121e+09,2023-10-24 00:21:24.331389,2023-10-24 04:21:24.331389+00:00,56,4.878704,11.478458,137.741497,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,124.569,2023,10,0,1,2023-10-24,21,3,2,1,2023-10-24 00:21:29.210093071,2023,10,2023-10-24,0,21,2,3,1,NaN,1850,56.647128,NaN,NaN,NaN,NaN,NaN
5,1.698121e+09,2023-10-24 00:22:03.749152,2023-10-24 04:22:03.749152+00:00,135,19.078445,7.076048,84.912580,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,116.638,2023,10,0,1,2023-10-24,22,3,2,1,2023-10-24 00:22:22.827597200,2023,10,2023-10-24,0,22,2,3,1,NaN,4206,1.406800,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,1.701996e+09,2023-12-07 19:31:41.216187,2023-12-08 00:31:41.216187+00:00,185,19.186214,9.642340,115.708082,NUM,4,3751,"""On the table of show bread they shall spread ...",7.0,132.705476,128.780425,133.934766,133.195,2023,12,19,1,2023-12-07,31,4,3,2,2023-12-07 19:32:00.402400800,2023,12,2023-12-07,19,32,3,4,2,0.0,3391,20.515826,10.810811,10.810811,NaN,NaN,NaN
4262,1.701996e+09,2023-12-07 19:32:01.025332,2023-12-08 00:32:01.025332+00:00,112,9.103476,12.302993,147.635916,NUM,4,3752,"They shall spread on them a scarlet cloth, and...",8.0,135.349267,128.831273,133.945498,133.198,2023,12,19,1,2023-12-07,32,4,3,2,2023-12-07 19:32:10.128807900,2023,12,2023-12-07,19,32,3,4,2,0.0,1089,74.490035,4.464286,4.464286,NaN,NaN,NaN
4263,1.701996e+09,2023-12-07 19:32:10.719891,2023-12-08 00:32:10.719891+00:00,168,14.232376,11.804072,141.648869,NUM,4,3753,"""They shall take a blue cloth and cover the la...",9.0,135.235827,128.737423,133.969192,133.200,2023,12,19,1,2023-12-07,32,4,3,2,2023-12-07 19:32:24.952267300,2023,12,2023-12-07,19,32,3,4,2,0.0,1544,63.821805,4.761905,4.761905,NaN,NaN,NaN


In [69]:
fig_df_results_by_test_number = px.line(df_results, x = df_results.index, 
y = ['WPM', 'Last 10 Avg', 'Last 100 Avg', 'Last 1000 Avg', 'cumulative_avg'])
fig_df_results_by_test_number.write_html('Analyses/results_by_test_number.html')
fig_df_results_by_test_number.write_image('Analyses/results_by_test_number.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_df_results_by_test_number

## Creating WPM histograms for (1) all tests and (2) the last 1000 tests:

(Until you've taken more than 1,000 tests, these histograms will have the same appearance.)

In [70]:
fig_wpm_histogram = px.histogram(x = df_results['WPM'], nbins = 50, 
text_auto = True)
fig_wpm_histogram.update_layout(bargroupgap = 0.1) # Adds a bit of space
# in between histogram bars. See https://stackoverflow.com/a/62925197/13097194
fig_wpm_histogram.write_html('Analyses/wpm_histogram.html')
fig_wpm_histogram.write_image('Analyses/wpm_histogram.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_wpm_histogram


In [71]:
fig_wpm_histogram = px.histogram(x = df_results.tail(1000)['WPM'], nbins = 50, 
text_auto = True)
fig_wpm_histogram.update_layout(bargroupgap = 0.1) # Adds a bit of space
# in between histogram bars. See https://stackoverflow.com/a/62925197/13097194
fig_wpm_histogram.write_html('Analyses/wpm_histogram_last_1000.html')
fig_wpm_histogram.write_image('Analyses/wpm_histogram_last_1000.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_wpm_histogram


## Evaluating average results by month:

In [72]:
df_results_by_month = df_results.pivot_table(
    index = ['Local_Start_Year', 'Local_Start_Month'], values = [
'Count', 'WPM'], aggfunc = {'Count':'sum', 'WPM':'mean'}).reset_index()
df_results_by_month['Year/Month'] = df_results_by_month[
    'Local_Start_Year'].astype('str') + '-' + df_results_by_month[
    'Local_Start_Month'].astype('str')
df_results_by_month.to_csv('Analyses/results_by_month.csv', index = False)
df_results_by_month

,Local_Start_Year,Local_Start_Month,Count,WPM,Year/Month
0,2023,10,332,135.657830,2023-10
1,2023,11,2863,132.415925,2023-11
2,2023,12,1070,134.577321,2023-12


In [73]:
fig_results_by_month = px.bar(df_results_by_month, x = 'Year/Month', 
y = 'WPM', color = 'Count', text_auto = '.6s')
fig_results_by_month.update_xaxes(type = 'category') # This line, based on
# Pracheta's response at https://stackoverflow.com/a/64424308/13097194,
# updates the axes to show the date-month pairs as strings rather than 
# as Plotly-formatted date values. This will also prevent missing
# months from appearing in the graph.
fig_results_by_month.write_html('Analyses/results_by_month.html')
fig_results_by_month.write_image('Analyses/results_by_month.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_results_by_month

## Evaluating average results by hour of day:

In [74]:
df_results_by_hour = df_results.pivot_table(index = ['Local_Start_Hour'], 
values = ['Count', 'WPM'], 
aggfunc = {'Count':'sum', 'WPM':'mean'}).reset_index()
df_results_by_hour.to_csv('Analyses/results_by_hour.csv')
df_results_by_hour

,Local_Start_Hour,Count,WPM
0,0,295,135.354986
1,1,42,131.637671
2,7,120,136.946364
3,8,152,130.189937
4,9,373,134.178345
5,10,58,134.531258
6,11,9,144.270436
7,12,1,76.853476
8,13,248,127.357054
9,14,7,120.537188


In [75]:
fig_results_by_hour = px.bar(df_results_by_hour, x = 'Local_Start_Hour', 
y = 'WPM', color = 'Count', text_auto = '.6s')
fig_results_by_hour.write_html('Analyses/results_by_hour.html')
fig_results_by_hour.write_image('Analyses/results_by_hour.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_results_by_hour

# Comparing mean WPMs by Bible books:

In [76]:
df_wpm_by_book = df_results.pivot_table(index = 'Book', values = 'WPM', 
aggfunc = ['count', 'mean'], margins = True, 
margins_name = 'Total').reset_index()
df_wpm_by_book.columns = 'Book', 'Tests', 'WPM'
df_wpm_by_book.sort_values('WPM', ascending = False, inplace = True)
df_wpm_by_book.reset_index(drop=True,inplace=True)
df_wpm_by_book.to_csv('Analyses/mean_wpm_by_book.csv')
df_wpm_by_book


,Book,Tests,WPM
0,REV,1,161.093106
1,2JN,4,151.783862
2,JUD,1,148.672335
3,3JN,15,147.630110
4,SNG,3,144.084300
5,2KI,1,143.531468
6,1JN,105,138.061187
7,LEV,859,136.759074
8,LUK,357,136.346558
9,Total,4265,133.210534


In [77]:
# The following chart will display a bar for each book for which at least one 
# test has been taken. It will also show a line that corresponds to the player's
# overall WPM across all books. The bars are colored by test count, making
# it easier to identify which bars might be skewed by a low number of results.
# The 'Total' value in df_wpm_by_book is displayed as a line instead of as
# a color so as not to interfere with the color gradient.

# Retrieving the total mean WPM value in df_wpm_by_book:
total_mean_wpm = df_wpm_by_book.query("Book == 'Total'").iloc[0]['WPM']
total_mean_wpm

fig_mean_wpm_by_book = px.bar(df_wpm_by_book.query("Book != 'Total'"), 
x = 'Book', y = 'WPM', color = 'Tests', text_auto = '.6s')
fig_mean_wpm_by_book.add_shape(type = 'line', x0 = -0.5, 
x1 = len(df_wpm_by_book) -1.5, y0 = total_mean_wpm, y1 = total_mean_wpm)
# See https://plotly.com/python/shapes/ for the add_shape() code.
# The use of -0.5 and len() - 1.5 is based on gleasocd's answer at 
# https://stackoverflow.com/a/40408960/13097194 . len(df) - 0.5 would normally
# work, except that I reduced the size of the DataFrame by 1 when excluding
# the 'Total' book.
fig_mean_wpm_by_book.write_html('Analyses/mean_wpm_by_book.html')
fig_mean_wpm_by_book.write_image('Analyses/mean_wpm_by_book.png', width = 1920, 
height = 1080, engine = 'kaleido', scale = 2)
fig_mean_wpm_by_book


## Comparing mean WPM for error-free and non-error free tests:

In [78]:
# We'll only create the graph if there is at least one mistake-free
# result and one result with errors. (Otherwise, we won't be able to
# compare these two outcomes.)
if (len(df_results.query("Mistake_Free_Test == 0")) >= 1) & (
    len(df_results.query("Mistake_Free_Test == 1")) >= 1):
    df_wpm_by_mistake_free_status = df_results.pivot_table(index = 'Mistake_Free_Test', values = 'WPM', aggfunc = 'mean').reset_index()
    df_wpm_by_mistake_free_status['Mistake_Free_Test'].replace({0:'No', 1:'Yes'}, inplace = True)
    df_wpm_by_mistake_free_status.to_csv('Analyses/wpm_by_mistake_free_status.csv', index = False)
    df_wpm_by_mistake_free_status

In [79]:
# We'll only create the graph if there is at least one mistake-free
# result and one result with errors. (Otherwise, we won't be able
# to compare these two outcomes.)
if (len(df_results.query("Mistake_Free_Test == 0")) >= 1) & (
    len(df_results.query("Mistake_Free_Test == 1")) >= 1):
    fig_wpm_by_mistake_free_status = px.bar(df_wpm_by_mistake_free_status, x = 'Mistake_Free_Test', y = 'WPM', text_auto = '.6s')
    fig_wpm_by_mistake_free_status.write_html('Analyses/mean_wpm_by_mistake_free_status.html')
    fig_wpm_by_mistake_free_status.write_image('Analyses/mean_wpm_by_mistake_free_status.png', width = 1920, 
    height = 1080, engine = 'kaleido', scale = 2)
    fig_wpm_by_mistake_free_status

In [80]:
df_results.head(3)

,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms)
Test_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023,10,0,1,2023-10-24,17,2,2,1,2023-10-24 00:17:44.588771456,2023,10,2023-10-24,0,17,2,2,1,NaN,3944,7.549824,NaN,NaN,NaN,NaN,NaN
2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:20.163038180,2023,10,2023-10-24,0,18,2,2,1,NaN,3388,20.586166,NaN,NaN,NaN,NaN,NaN
3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:29.095581096,2023,10,2023-10-24,0,18,2,2,1,NaN,1464,65.697538,NaN,NaN,NaN,NaN,NaN


### Evaluating the relationship between backspaces as a % of verse length and WPM:

In [81]:
fig_backspaces_wpm_scatter = px.scatter(df_results, 
x = 'Backspaces as % of Verse Length', y = 'WPM', trendline = 'ols')
# Note that you can hover over the best fit line to see the regression results.
fig_backspaces_wpm_scatter.write_html('Analyses/accuarcy_wpm_scatter.html')
fig_backspaces_wpm_scatter.write_image('Analyses/accuarcy_wpm_scatter.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_backspaces_wpm_scatter

In [82]:
fig_accuracy_wpm_histogram = px.histogram(df_results, x = 'Backspaces as % of Verse Length', y = 'WPM', histfunc = 'avg', nbins = 20, text_auto = '.6s')
fig_accuracy_wpm_histogram.update_layout(bargroupgap = 0.1)
fig_accuracy_wpm_histogram.write_html('Analyses/accuracy_wpm_histogram.html')
fig_accuracy_wpm_histogram.write_image('Analyses/accuracy_wpm_histogram.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_accuracy_wpm_histogram


In [83]:
fig_accuracy_count_histogram = px.histogram(df_results, x = 'Backspaces as % of Verse Length', nbins = 20, text_auto = True)
fig_accuracy_count_histogram.update_layout(bargroupgap = 0.1)
fig_accuracy_count_histogram.write_html('Analyses/accuracy_count_histogram.html')
fig_accuracy_count_histogram.write_image('Analyses/accuracy_count_histogram.png', 
width = 1920, height = 1080, engine = 'kaleido', scale = 2)
fig_accuracy_count_histogram

In [84]:
df_wpm_by_percentile = df_results['WPM'].quantile(q = np.arange(0, 1.05, 0.05)).transpose().reset_index()
df_wpm_by_percentile.columns = ['Percentile', 'WPM']
df_wpm_by_percentile['Percentile'] = (
df_wpm_by_percentile['Percentile']*100).astype('int')
# Some of the WPM values had additional decimal values (e.g. 15.000000000000002,
# might currently have additional decimal values, so rounding them as integers
# helps simplify them.
df_wpm_by_percentile.to_csv('Analyses/wpm_by_percentile.csv', index = False)
df_wpm_by_percentile

,Percentile,WPM
0,0,41.385335
1,5,97.522112
2,10,105.539252
3,15,111.088727
4,20,115.401776
5,25,119.094719
6,30,122.471673
7,35,125.575776
8,40,128.174109
9,45,131.037829


In [85]:
fig_wpm_by_percentile = px.bar(df_wpm_by_percentile, x = 'Percentile', y = 'WPM', text_auto = '.6s')
fig_wpm_by_percentile.update_xaxes(type = 'category')
fig_wpm_by_percentile.write_html('Analyses/wpm_by_percentile.html')
fig_wpm_by_percentile.write_image('Analyses/wpm_by_percentile.png', width = 1920, 
height = 1080, engine = 'kaleido', scale = 2)
fig_wpm_by_percentile

# Creating word-level analyses:

Note: This section used to include code that would calculate the fastest single-word entry times. However, I found that these results could be distorted by computer lag. Specifically, if the player enters keys while the computer is frozen, some of those keypress durations will be reported as shorter than they actually were. (This makes sense, in that while the computer is frozen, the actual keypress time might not get logged; instead, a later post-freeze time will get logged. This post-freeze time will then be closer to the next keypress time, causing a shorter-than-actual keypress time to get reported.) (I simulated computer lag in my tests by entering a time.sleep() command, but I believe actual lag would produce similar errors). These skewed keypress durations can in turn skew the reported time needed to enter each word.

Therefore, while the code still shows median word entry times (which are less susceptible to this issue), it will no longer calculate the fastest times that a particular word was entered. These can still be located within the word stats file, but the results should be interpreted with great caution.

In [86]:
# We'll skip the word-level analyses section if we have no data to analyze.

if len(df_word_stats) == 0:
    run_word_analyses = 0
else:
    run_word_analyses = 1
    print("Creating word-level analyses.")

run_word_analyses

1

In [87]:
if run_word_analyses == 1:
    df_word_stats_pivot = df_word_stats.pivot_table(index = 'word', values = {'Count', 'typed_word_without_mistakes', 'WPM'}, aggfunc = {'Count':'sum', 'typed_word_without_mistakes':'mean', 'WPM':'median'}).reset_index().sort_values('WPM', ascending = False).reset_index(drop=True)
    df_word_stats_pivot.rename(columns = {'WPM':'Median WPM'}, inplace = True)
    df_common_word_stats_pivot = df_word_stats_pivot.query("Count >= 5").copy()
    df_word_stats_pivot.to_csv('Analyses/word_stats_pivot.csv', index = False)
    df_common_word_stats_pivot.to_csv('Analyses/word_stats_pivot_common.csv', 
    index = False)
    df_word_stats_pivot


In [88]:
df_common_word_stats_pivot

,word,Count,Median WPM,typed_word_without_mistakes
2,year,8,263.643317,0.875000
5,things,9,251.374618,1.000000
7,eat,11,248.766533,0.909091
11,around,8,235.696922,0.875000
13,about,5,232.055712,1.000000
...,...,...,...,...
1215,If,35,66.503806,0.942857
1221,requirements,5,65.160515,0.200000
1233,Naphtali,5,58.850228,0.400000
1235,A,7,58.667571,1.000000


In [89]:
if run_word_analyses == 1:
    fig_words_with_highest_median_wpms = px.bar(df_word_stats_pivot.head(100), 
    x = 'word', y = 'Median WPM', color = 'Count', text_auto = '.6s')
    fig_words_with_highest_median_wpms.write_html('Analyses/words_with_highest_median_wpms.html')
    fig_words_with_highest_median_wpms.write_image('Analyses/words_with_highest_median_wpms.png', width = 1920, 
    height = 1080, engine = 'kaleido', scale = 2)
    fig_words_with_highest_median_wpms

In [90]:
if (run_word_analyses == 1) & (len(df_common_word_stats_pivot) >= 1):
    fig_words_with_highest_median_wpms_common = px.bar(df_common_word_stats_pivot.head(100), 
    x = 'word', y = 'Median WPM', color = 'Count', text_auto = '.6s')
    fig_words_with_highest_median_wpms_common.write_html('Analyses/words_with_highest_median_wpms_common.html')
    fig_words_with_highest_median_wpms_common.write_image('Analyses/words_with_highest_median_wpms_common.png', width = 1920, 
    height = 1080, engine = 'kaleido', scale = 2)
    fig_words_with_highest_median_wpms_common

In [91]:
if run_word_analyses == 1:
    fig_words_with_lowest_median_wpms = px.bar(
    df_word_stats_pivot.sort_values(
    'Median WPM').head(100), x = 'word', y = 'Median WPM', color = 'Count', text_auto = '.6s')
    fig_words_with_lowest_median_wpms.write_html(
    'Analyses/words_with_lowest_median_wpms.html')
    fig_words_with_lowest_median_wpms.write_image(
    'Analyses/words_with_lowest_median_wpms.png', width = 1920, 
    height = 1080, engine = 'kaleido', scale = 2)
    fig_words_with_lowest_median_wpms

In [92]:
if (run_word_analyses == 1) & (len(df_common_word_stats_pivot) >= 1):
    fig_words_with_lowest_median_wpms_common = px.bar(
    df_common_word_stats_pivot.sort_values('Median WPM').head(100), 
    x = 'word', y = 'Median WPM', color = 'Count', text_auto = '.6s')
    fig_words_with_lowest_median_wpms_common.write_html('Analyses/words_with_lowest_median_wpms_common.html')
    fig_words_with_lowest_median_wpms_common.write_image('Analyses/words_with_lowest_median_wpms_common.png', width = 1920, 
    height = 1080, engine = 'kaleido', scale = 2)
    fig_words_with_lowest_median_wpms_common

In [93]:
if (run_word_analyses == 1) & (len(df_common_word_stats_pivot) >= 1):
    fig_words_with_highest_accuracy_rates_common = px.bar(
    df_common_word_stats_pivot.sort_values(
        'typed_word_without_mistakes', ascending = False).head(100),
    x = 'word', y = 'typed_word_without_mistakes', color = 'Count', text_auto = '.6s')
    fig_words_with_highest_accuracy_rates_common.write_html('Analyses/words_with_highest_accuracy_rates_common.html')
    fig_words_with_highest_accuracy_rates_common.write_image('Analyses/words_with_highest_accuracy_rates_common.png', width = 1920, 
    height = 1080, engine = 'kaleido', scale = 2)
    fig_words_with_highest_accuracy_rates_common

In [94]:
if (run_word_analyses == 1) & (len(df_common_word_stats_pivot) >= 1):
    fig_words_with_lowest_accuracy_rates_common = px.bar(
    df_common_word_stats_pivot.sort_values(
        'typed_word_without_mistakes').head(100),
    x = 'word', y = 'typed_word_without_mistakes', color = 'Count', text_auto = '.3s')
    fig_words_with_lowest_accuracy_rates_common.write_html('Analyses/words_with_lowest_accuracy_rates_common.html')
    fig_words_with_lowest_accuracy_rates_common.write_image('Analyses/words_with_lowest_accuracy_rates_common.png', width = 1920, 
    height = 1080, engine = 'kaleido', scale = 2)
    fig_words_with_lowest_accuracy_rates_common

This script used to show the highest-ever WPMs for individual words, but this code has been removed due to accuracy issues.

In [95]:
analysis_end_time = time.time()
analysis_time = analysis_end_time - analysis_start_time
print(f"Finished updating analyses in {round(analysis_time, 3)} seconds. \
Enter any key to exit.") # Allows the console to stay open when the
# .py version of the program is run

input()

Finished updating analyses in 41.883 seconds. Enter any key to exit.


''